# Integration: MaxFuse and MARIO

This notebook performs cross-modal integration of RNA-seq and CODEX protein data.

## Recommended Workflow

**For cross-modal data (RNA + Protein/CODEX):**
1. Run Steps 1-4 (data loading and preparation)
2. **Skip** MARIO section (designed for same-modality data)
3. Run **MaxFuse Integration** (Step 7+)

**For same-modality data (e.g., CITE-seq + CyTOF):**
1. Run all steps including MARIO

## What Each Method Does

| Method | Best For | Key Feature |
|--------|----------|-------------|
| **MaxFuse** | Cross-modal (RNA↔Protein) | Handles weak feature linkage |
| **MARIO** | Same-modality (Protein↔Protein) | Statistical matchability test |


In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import json
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.io import mmread

# MaxFuse imports
import maxfuse as mf
from maxfuse import Fusor, Mario
from maxfuse.mario import pipelined_mario

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load preprocessed data from 1_preprocessing.ipynb
# Run 1_preprocessing.ipynb first to generate these files

import os

results_dir = 'results/1_preprocessing'

if not os.path.exists(results_dir):
    raise FileNotFoundError(
        f"Results directory '{results_dir}' not found. "
        f"Run 1_preprocessing.ipynb first to generate the input files."
    )

# Load processed AnnData objects
protein_adata = sc.read_h5ad(f'{results_dir}/protein_adata.h5ad')
rna_adata = sc.read_h5ad(f'{results_dir}/rna_adata.h5ad')
rna_adata_lognorm = sc.read_h5ad(f'{results_dir}/rna_adata_lognorm.h5ad')

print(f"Loaded from {results_dir}/")
print(f"  Protein data: {protein_adata.shape}")
print(f"  RNA data: {rna_adata.shape}")
print(f"  RNA log-normalized: {rna_adata_lognorm.shape}")

# Load preprocessing parameters
with open(f'{results_dir}/preprocessing_params.json', 'r') as f:
    preprocess_params = json.load(f)
print(f"\nPreprocessing timestamp: {preprocess_params['timestamp']}")

Loaded from results/1_preprocessing/
  Protein data: (1900754, 59)
  RNA data: (1669, 16050)
  RNA log-normalized: (1669, 16050)

Preprocessing timestamp: 2026-01-15T14:43:38.389358


## Step 3: Build Protein-Gene Correspondence

Map CODEX protein markers to their corresponding gene names in the RNA data.

In [3]:
# # Load correspondence table
correspondence = pd.read_csv('data/protein_gene_conversion.csv', encoding='utf-8-sig')
print(f"Correspondence table: {correspondence.shape[0]} entries")
correspondence.head(10)

Correspondence table: 375 entries


,Protein name,RNA name
0,CD80,CD80
1,CD86,CD86
2,CD274,CD274
3,CD273,PDCD1LG2
4,CD275,ICOSLG
5,CD275-1,ICOSLG
6,CD275-2,ICOSLG
7,CD11b,ITGAM
8,CD11b-1,ITGAM
9,CD11b-2,ITGAM


In [4]:
# Find matching features between CODEX markers and RNA genes
rna_protein_correspondence = []
unmatched_proteins = []

for marker in protein_adata.var_names:
    # Skip DAPI and ECAD (not useful for cell type matching), also add any to skip due to failed staining
    if marker in ['DAPI', 'ECAD', 'E-cadherin', 'IAPP', 'LAG3']:
        continue
    
    # Look up in correspondence table
    matches = correspondence[correspondence['Protein name'].str.lower() == marker.lower()]
    
    if len(matches) == 0:
        # Try alternative names
        alt_names = {
            'CD3e': 'CD3',
            'FoxP3': 'FOXP3',
            'HLADR': 'HLA-DR',
            'Lyve1': 'LYVE1',
            'SMActin': 'aSMA',
            'CollagenIV': 'collagen IV',
        }
        alt_marker = alt_names.get(marker, marker)
        matches = correspondence[correspondence['Protein name'].str.lower() == alt_marker.lower()]
    
    if len(matches) > 0:
        rna_names_str = matches.iloc[0]['RNA name']
        if 'Ignore' in str(rna_names_str):
            unmatched_proteins.append((marker, 'Ignored'))
            continue
        
        # Try each RNA name option
        found = False
        for rna_name in str(rna_names_str).split('/'):
            if rna_name in rna_adata.var_names:
                rna_protein_correspondence.append([rna_name, marker])
                found = True
                break
        if not found:
            unmatched_proteins.append((marker, rna_names_str))
    else:
        unmatched_proteins.append((marker, 'Not in table'))

rna_protein_correspondence = np.array(rna_protein_correspondence)
print(f"Found {len(rna_protein_correspondence)} protein-gene pairs")

if unmatched_proteins:
    print(f"\nUnmatched proteins ({len(unmatched_proteins)}):")
    for prot, reason in unmatched_proteins:
        print(f"  {prot}: {reason}")

Found 54 protein-gene pairs

Unmatched proteins (1):
  Pan-Cytokeratin: Ignored


In [5]:
# Remove duplicates (same RNA mapping to multiple proteins)
# Keep first occurrence
seen_rna = set()
unique_pairs = []
for rna, prot in rna_protein_correspondence:
    if rna not in seen_rna:
        seen_rna.add(rna)
        unique_pairs.append([rna, prot])
    else:
        print(f"Removing duplicate RNA mapping: {rna} -> {prot}")

rna_protein_correspondence = np.array(unique_pairs)
print(f"\nFinal correspondence: {len(rna_protein_correspondence)} pairs")

print("\nMatched features:")
for rna, prot in rna_protein_correspondence:
    print(f"  {rna:15} <-> {prot}")


Final correspondence: 54 pairs

Matched features:
  LAMP1           <-> CD107a
  CD4             <-> CD4
  PECAM1          <-> CD31
  ACTA2           <-> SMA
  CD68            <-> CD68
  CD44            <-> CD44
  VIM             <-> Vimentin
  CD99            <-> CD99
  IDO1            <-> IDO1
  CEACAM1         <-> CD66
  INS             <-> INS
  KRT8            <-> Ker8-18
  ITGAX           <-> CD11c
  CD38            <-> CD38
  HLA-DRA         <-> HLA-DR
  CD34            <-> CD34
  NOS2            <-> iNOS
  LGALS3          <-> M2Gal3
  TUBB3           <-> B3TUBB
  CD8A            <-> CD8
  PCNA            <-> PCNA
  FOXP3           <-> FOXP3
  B3GAT1          <-> CD57
  MKI67           <-> Ki67
  GZMB            <-> Granzyme B
  HLA-A           <-> HLA-A
  MS4A1           <-> CD20
  COL4A1          <-> Collagen IV
  VSIR            <-> VISTA
  PDCD1           <-> PD-1
  SST             <-> SST
  TCF7            <-> TCF-1
  TOX             <-> TOX
  CAV1            <-> Caveolin


## Step 4: Prepare Arrays for Integration

Extract and normalize:
- **Shared arrays**: Corresponding protein/gene features (used for initial matching)
- **Active arrays**: All features (used for refinement)

In [6]:
# Extract shared features from FILTERED data
rna_shared_adata = rna_adata[:, rna_protein_correspondence[:, 0]].copy()
protein_shared_adata = protein_adata[:, rna_protein_correspondence[:, 1]].copy()

print(f"rna_shared_adata: {rna_shared_adata.shape}")
print(f"protein_shared_adata: {protein_shared_adata.shape}")

rna_shared_adata: (1669, 54)
protein_shared_adata: (1900754, 54)


In [7]:
# Normalize shared features
# 
# Protein: GATED data is already z-scored (zero-mean, unit variance)
# RNA: Apply log1p then z-score to match

from scipy.stats import rankdata, norm, zscore
import os

# ============================================================
# LOAD DATA
# ============================================================
print("="*70)
print("LOADING DATA")
print("="*70)

# Load gated protein data (already z-scored)
gated_codex_path = 'data/6551_leiden_umap.h5ad'
protein_gated = sc.read_h5ad(gated_codex_path)
print(f"Gated Protein: {protein_gated.shape[0]:,} cells, {protein_gated.shape[1]} markers")
print(f"  Mean: {protein_gated.X.mean():.3f}, Std: {protein_gated.X.std():.3f}")
print(f"  (Confirmed: zero-mean, unit variance)")

# Find shared features
conversion = pd.read_csv('data/protein_gene_conversion.csv')
rna_genes = set(rna_adata.var_names)
protein_markers = set(protein_gated.var_names)

shared = []
for _, row in conversion.iterrows():
    prot = row['Protein name']
    rna = str(row['RNA name'])
    if pd.isna(prot) or prot not in protein_markers:
        continue
    if rna.startswith('Ignore'):
        continue
    for gene in [g.strip() for g in rna.split('/')]:
        if gene in rna_genes:
            shared.append((gene, prot))
            break

rna_protein_correspondence = np.array(shared)
print(f"\nShared features: {len(shared)}")

# Subset to shared features
rna_shared_adata = rna_adata[:, [s[0] for s in shared]].copy()
protein_shared_adata = protein_gated[:, [s[1] for s in shared]].copy()

print(f"  RNA: {rna_shared_adata.shape}")
print(f"  Protein: {protein_shared_adata.shape}")

# ============================================================
# GET RAW ARRAYS
# ============================================================

rna_shared_raw = rna_shared_adata.X.copy()
if sparse.issparse(rna_shared_raw):
    rna_shared_raw = rna_shared_raw.toarray()

protein_shared_raw = protein_shared_adata.X.copy()
if sparse.issparse(protein_shared_raw):
    protein_shared_raw = protein_shared_raw.toarray()

# ============================================================
# NORMALIZE RNA: log1p -> z-score (to match protein)
# ============================================================
print("\n" + "="*70)
print("RNA NORMALIZATION: log1p -> z-score")
print("="*70)

# Library size normalization
sc.pp.normalize_total(rna_shared_adata, target_sum=1e4)

# Log1p
sc.pp.log1p(rna_shared_adata)
rna_after_log = rna_shared_adata.X.copy()
if sparse.issparse(rna_after_log):
    rna_after_log = rna_after_log.toarray()

print(f"After log1p - mean: {rna_after_log.mean():.3f}, std: {rna_after_log.std():.3f}")

# Z-score per feature (to match protein's zero-mean unit variance)
rna_zscore = np.zeros_like(rna_after_log)
for j in range(rna_after_log.shape[1]):
    col = rna_after_log[:, j]
    if col.std() > 0:
        rna_zscore[:, j] = (col - col.mean()) / col.std()
    else:
        rna_zscore[:, j] = 0

print(f"After z-score - mean: {rna_zscore.mean():.3f}, std: {rna_zscore.std():.3f}")

# ============================================================
# PROTEIN: Already z-scored, use as-is
# ============================================================
print("\n" + "="*70)
print("PROTEIN: Already z-scored (use as-is)")
print("="*70)

protein_zscore = protein_shared_raw.copy()
print(f"Protein - mean: {protein_zscore.mean():.3f}, std: {protein_zscore.std():.3f}")

# ============================================================
# STORE RESULTS
# ============================================================

# Final normalized arrays
rna_shared = rna_zscore
protein_shared = protein_zscore

# For compatibility with downstream code
rna_shared_after_log = rna_after_log
rna_shared_after_scale = rna_zscore
protein_shared_after_arcsinh = protein_zscore
protein_shared_after = protein_zscore

# Update AnnData objects
rna_shared_adata.X = rna_zscore.astype(np.float32)
protein_shared_adata.X = protein_zscore.astype(np.float32)

# ============================================================
# VERIFICATION
# ============================================================
print("\n" + "="*70)
print("VERIFICATION")
print("="*70)

print(f"\n{'Modality':<10} {'Cells':>12} {'Features':>10} {'Mean':>10} {'Std':>10}")
print("-"*55)
print(f"{'RNA':<10} {rna_shared.shape[0]:>12,} {rna_shared.shape[1]:>10} {rna_shared.mean():>10.3f} {rna_shared.std():>10.3f}")
print(f"{'Protein':<10} {protein_shared.shape[0]:>12,} {protein_shared.shape[1]:>10} {protein_shared.mean():>10.3f} {protein_shared.std():>10.3f}")

# Per-feature stats
print("\nPer-feature statistics:")
print(f"{'Feature':<15} {'RNA mean':>10} {'RNA std':>10} {'Prot mean':>10} {'Prot std':>10}")
print("-"*60)
for i, (rna_name, prot_name) in enumerate(rna_protein_correspondence[:10]):
    r_mean, r_std = rna_shared[:, i].mean(), rna_shared[:, i].std()
    p_mean, p_std = protein_shared[:, i].mean(), protein_shared[:, i].std()
    print(f"{prot_name[:14]:<15} {r_mean:>10.3f} {r_std:>10.3f} {p_mean:>10.3f} {p_std:>10.3f}")
print("...")

print("\n" + "="*70)
print("SUCCESS: Both modalities are now z-scored (zero-mean, unit variance)")
print("="*70)


LOADING GATED PROTEIN DATA
Loaded GATED CODEX: 1,213,219 cells, 59 markers
  Data range: [-10.50, 12.59]
  Detection: value > 0 (data is already gated/scaled)

Shared features: 54
  RNA: (1669, 54)
  Protein: (1213219, 54)
DETECTION-AWARE NORMALIZATION PIPELINE
Cell counts: RNA = 1,669, Protein = 1,213,219
STEP 1: RNA transformation (log1p)
RNA raw - zeros: 88.7%
After log1p - mean: 0.712
STEP 2: Protein data (already gated/scaled)
Protein scaled data:
  Range: [-10.50, 12.59]
  Mean: 0.306
Detection rates (value > 0):
  Marker                 Detected %
  -----------------------------------
  CD11b                       54.9%
  CD4                         76.2%
  CD8                         80.0%
  CD56                        64.8%
  CD11c                       73.8%
  CD34                        34.1%
  CD44                        63.1%
  CD20                        78.1%
  CD31                        50.8%
  Podoplanin                  73.1%
  CD107a                      65.8%
  HLA

In [ ]:
# Load raw CODEX data for diagnostic cells (background detection analysis)
# This requires the original TSV file from preprocessing

import os
codex_tsv_path = 'data/6551_cells.csv'

if os.path.exists(codex_tsv_path):
    codex_df = pd.read_csv(codex_tsv_path)
    print(f"Loaded raw CODEX data: {codex_df.shape}")
    print(f"Columns: {len(codex_df.columns)}")
    CODEX_RAW_AVAILABLE = True
else:
    print(f"Raw CODEX file not found at: {codex_tsv_path}")
    print("Skipping background detection diagnostic cells.")
    print("These cells are optional - the integration will work without them.")
    codex_df = None
    CODEX_RAW_AVAILABLE = False

In [ ]:
# Diagnostic: Background detection using Cell Median
# Skip if raw CODEX data not available

if not CODEX_RAW_AVAILABLE:
    print("Skipping: Raw CODEX data not available (run from preprocessing to analyze)")
else:
    # Diagnostic: Background detection using Cell Median
    # For high-background CODEX data, we need DATA-DRIVEN per-marker thresholds
    # instead of assuming background = 0
    
    print("="*70)
    print("PROTEIN BACKGROUND DETECTION (using Cell Median with per-marker thresholds)")
    print("="*70)
    
    protein_names = list(rna_protein_correspondence[:, 1])
    n_features = len(protein_names)
    
    # We need the median values from the original CODEX dataframe
    # Build mapping from protein name to column
    protein_to_median_col = {}
    for prot in protein_names:
        # Find the matching column in codex_df
        for col in codex_df.columns:
            if prot in col and 'Cell:' in col and 'Median' in col:
                protein_to_median_col[prot] = col
                break
    
    print(f"Analyzing {n_features} shared protein features...")
    print("Using DATA-DRIVEN per-marker thresholds based on 25th percentile")
    print("Formula: threshold = max(p25 * 0.5, 0.5)")
    print("  - For CyTOF-like data (p25~0): threshold ~0.5 (original behavior)")
    print("  - For high-background CODEX (p25~150): threshold ~75")
    
    # Store detection info
    detection_stats = {}
    marker_thresholds = {}  # Store per-marker thresholds
    
    # Create figure: each protein gets a panel with Mean and Median histograms
    plots_per_row = 4
    n_feature_rows = int(np.ceil(n_features / plots_per_row))
    fig, axes = plt.subplots(n_feature_rows, plots_per_row, figsize=(16, 3.5*n_feature_rows))
    if n_feature_rows == 1:
        axes = axes.reshape(1, -1)
    axes = axes.flatten()
    
    print(f"{'Protein':<12} {'p25':>8} {'threshold':>10} {'Below':>12} {'Above':>12} {'% Detected':>12}")
    print("-"*70)
    
    for i in range(n_features):
        prot_name = protein_names[i]
        ax = axes[i]
        
        if prot_name in protein_to_median_col:
            median_col = protein_to_median_col[prot_name]
            median_vals = codex_df[median_col].values
            
            # Get corresponding mean column for comparison
            mean_col = median_col.replace('Median', 'Mean')
            mean_vals = codex_df[mean_col].values if mean_col in codex_df.columns else None
            
            # DATA-DRIVEN threshold based on 25th percentile
            # This adapts to the data scale:
            # - CyTOF (low values, p25~0): threshold ~0.5 (original behavior)
            # - CODEX with high background (p25~100-200): threshold ~50-100
            p25 = np.percentile(median_vals, 25)
            threshold = max(p25 * 0.5, 0.5)  # At least 0.5 for numerical stability
            marker_thresholds[prot_name] = threshold
            
            # Classification: Below threshold is background
            is_background = median_vals <= threshold
            is_detected = ~is_background
            
            n_background = is_background.sum()
            n_detected = is_detected.sum()
            pct_detected = 100 * n_detected / len(median_vals)
            
            detection_stats[prot_name] = {
                'n_background': n_background,
                'n_detected': n_detected,
                'pct_detected': pct_detected,
                'threshold': threshold,
                'p25': p25
            }
            
            print(f"{prot_name:<12} {p25:>8.1f} {threshold:>10.1f} {n_background:>12,} {n_detected:>12,} {pct_detected:>11.1f}%")
            
            # Plot: Histogram of Median values
            # Zoom to show distribution around threshold
            max_plot = max(threshold * 3, 50)
            median_zoomed = median_vals[median_vals <= max_plot]
            bins = np.linspace(0, max_plot, 50)
            ax.hist(median_zoomed, bins=bins, alpha=0.7, 
                    color='darkorange', edgecolor='white', linewidth=0.5)
            ax.axvline(x=threshold, color='green', linestyle='--', linewidth=2, 
                       label=f'threshold={threshold:.0f}')
            ax.set_title(f'{prot_name} ({pct_detected:.0f}% det)', fontsize=10)
            ax.set_xlabel('Cell Median')
            ax.legend(fontsize=7)
        else:
            print(f"{prot_name:<12} {'column not found':>56}")
            ax.set_visible(False)
    
    # Hide unused axes
    for j in range(n_features, len(axes)):
        axes[j].set_visible(False)
    
    plt.suptitle('Cell Median Distributions with per-marker thresholds', fontsize=12, y=1.01)
    plt.tight_layout()
    plt.show()
    
    # Summary
    avg_threshold = np.mean(list(marker_thresholds.values()))
    avg_detection = np.mean([s['pct_detected'] for s in detection_stats.values()])
    
    print(f"{'='*70}")
    print("SUMMARY")
    print("="*70)
    print(f"Background criterion: Cell Median <= per-marker threshold")
    print(f"Formula: threshold = max(25th_percentile * 0.5, 0.5)")
    print(f"Average threshold: {avg_threshold:.1f}")
    print(f"Average detection rate: {avg_detection:.1f}%")
    print(f"This adapts to high-background CODEX data where 'zero' isn't meaningful.")

In [ ]:
# Visualize DETECTION-AWARE normalization
# Skip if raw CODEX data not available

if not CODEX_RAW_AVAILABLE:
    print("Skipping: Raw CODEX data not available (run from preprocessing to analyze)")
else:
    # Visualize DETECTION-AWARE normalization
    # Updated to use Cell Median-based detection with DATA-DRIVEN per-marker thresholds
    
    fig, axes = plt.subplots(4, 4, figsize=(20, 16))
    
    feature_names = list(rna_protein_correspondence[:, 0])
    protein_names = list(rna_protein_correspondence[:, 1])
    n_features = rna_shared_raw.shape[1]
    
    # Define x_pos and width for bar charts
    x_pos = np.arange(n_features)
    width = 0.35
    
    # Pick a feature with moderate expression
    detection_rates_rna = [(rna_shared_raw[:, i] > 0).mean() for i in range(n_features)]
    good_features = [i for i, d in enumerate(detection_rates_rna) if 0.2 < d < 0.8]
    if good_features:
        best_feat_idx = good_features[len(good_features)//2]
    else:
        best_feat_idx = np.argmax(detection_rates_rna)
    feat_name = feature_names[best_feat_idx]
    prot_name = protein_names[best_feat_idx]
    
    # Build protein detection rates from Cell Median with DATA-DRIVEN thresholds
    protein_to_median_col = {}
    for prot in protein_names:
        for col in codex_df.columns:
            if prot in col and 'Cell:' in col and 'Median' in col:
                protein_to_median_col[prot] = col
                break
    
    # Calculate per-marker thresholds: threshold = max(p25 * 0.5, 0.5)
    marker_thresholds = {}
    for pname in protein_names:
        if pname in protein_to_median_col:
            median_vals = codex_df[protein_to_median_col[pname]].values
            p25 = np.percentile(median_vals, 25)
            marker_thresholds[pname] = max(p25 * 0.5, 0.5)
        else:
            marker_thresholds[pname] = 0.5  # Default fallback
    
    # Calculate detection rates: RNA uses >0, Protein uses Cell Median > per-marker threshold
    rna_det_rates = [(rna_shared_raw[:, i] > 0).mean() * 100 for i in range(n_features)]
    prot_det_rates = []
    for i, pname in enumerate(protein_names):
        if pname in protein_to_median_col:
            median_vals = codex_df[protein_to_median_col[pname]].values
            threshold = marker_thresholds[pname]
            prot_det_rates.append((median_vals > threshold).mean() * 100)
        else:
            # Fallback to raw >0
            prot_det_rates.append((protein_shared_raw[:, i] > 0).mean() * 100)
    
    rna_det = rna_det_rates[best_feat_idx]
    prot_det = prot_det_rates[best_feat_idx]
    prot_threshold = marker_thresholds.get(prot_name, 0.5)
    
    print(f"Example feature: {feat_name} / {prot_name}")
    print(f"  RNA detection: {rna_det:.1f}% ({int(rna_det/100 * rna_shared_raw.shape[0]):,} cells)")
    print(f"  Protein detection (Cell Median > {prot_threshold:.0f}): {prot_det:.1f}% ({int(prot_det/100 * protein_shared_raw.shape[0]):,} cells)")
    
    ZERO_VALUE = -2.5  # Must match value used in normalization
    
    # ============================================================
    # Row 1: RNA transformation pipeline
    # ============================================================
    ax = axes[0, 0]
    raw_vals = rna_shared_raw[:, best_feat_idx]
    ax.hist(raw_vals, bins=50, alpha=0.7, color='steelblue', edgecolor='white')
    ax.set_title(f'RNA: Raw Counts\n(zeros: {(raw_vals==0).mean()*100:.0f}%)', fontsize=10)
    ax.set_xlabel('Count')
    ax.set_ylabel('Cells')
    
    ax = axes[0, 1]
    log_vals = rna_shared_after_log[:, best_feat_idx]
    zeros = log_vals == 0
    nonzeros = ~zeros
    ax.hist(log_vals[nonzeros], bins=30, alpha=0.7, color='coral', edgecolor='white', label=f'Non-zero ({nonzeros.sum():,})')
    if zeros.sum() > 0:
        ax.axvline(x=0, color='gray', linestyle='--', linewidth=2, label=f'Zeros ({zeros.sum():,})')
    ax.set_title('RNA: After log1p\n(zeros at 0)', fontsize=10)
    ax.set_xlabel('log1p(count)')
    ax.legend(fontsize=8)
    
    ax = axes[0, 2]
    norm_vals = rna_shared_after_scale[:, best_feat_idx]
    nonzero_mask = norm_vals > ZERO_VALUE + 0.1
    ax.hist(norm_vals[nonzero_mask], bins=30, alpha=0.7, color='forestgreen', edgecolor='white', 
            label=f'Detected ({nonzero_mask.sum():,})')
    ax.axvline(x=ZERO_VALUE, color='red', linestyle='--', linewidth=2, 
               label=f'Not detected ({(~nonzero_mask).sum():,})')
    ax.set_title('RNA: Detection-Aware Normalized\n(zeros → fixed value)', fontsize=10)
    ax.set_xlabel('Normalized value')
    ax.legend(fontsize=8)
    
    ax = axes[0, 3]
    ax.axis('off')
    ax.text(0.1, 0.85, 'RNA Pipeline:', fontsize=12, fontweight='bold', transform=ax.transAxes)
    ax.text(0.1, 0.70, '1. normalize_total (library size)', fontsize=10, transform=ax.transAxes)
    ax.text(0.1, 0.58, '2. log1p (variance stabilization)', fontsize=10, transform=ax.transAxes)
    ax.text(0.1, 0.46, '3. Detection-aware normalization:', fontsize=10, transform=ax.transAxes)
    ax.text(0.15, 0.34, '• Non-zeros → rank → normal quantiles', fontsize=9, transform=ax.transAxes)
    ax.text(0.15, 0.22, f'• Zeros → fixed value ({ZERO_VALUE})', fontsize=9, transform=ax.transAxes)
    
    # ============================================================
    # Row 2: Protein transformation pipeline
    # ============================================================
    ax = axes[1, 0]
    raw_vals = protein_shared_raw[:, best_feat_idx]
    # Show Cell Median-based detection with per-marker threshold
    if prot_name in protein_to_median_col:
        median_vals = codex_df[protein_to_median_col[prot_name]].values
        threshold = marker_thresholds[prot_name]
        is_bg = median_vals <= threshold
        pct_bg = is_bg.mean() * 100
    else:
        pct_bg = (raw_vals == 0).mean() * 100
    ax.hist(raw_vals, bins=50, alpha=0.7, color='darkorange', edgecolor='white')
    ax.set_title(f'Protein: Raw MFI\n(background: {pct_bg:.0f}% via Cell Median)', fontsize=10)
    ax.set_xlabel('Mean Fluorescence Intensity')
    ax.set_ylabel('Cells')
    
    ax = axes[1, 1]
    arcsinh_vals = protein_shared_after_arcsinh[:, best_feat_idx]
    # Use Cell Median with per-marker threshold for zero classification
    if prot_name in protein_to_median_col:
        median_vals = codex_df[protein_to_median_col[prot_name]].values
        threshold = marker_thresholds[prot_name]
        zeros = median_vals <= threshold
    else:
        zeros = protein_shared_raw[:, best_feat_idx] == 0
    nonzeros = ~zeros
    ax.hist(arcsinh_vals[nonzeros], bins=30, alpha=0.7, color='purple', edgecolor='white', label=f'Detected ({nonzeros.sum():,})')
    if zeros.sum() > 0:
        # Show background cells as a separate histogram
        ax.hist(arcsinh_vals[zeros], bins=30, alpha=0.5, color='gray', edgecolor='white', label=f'Background ({zeros.sum():,})')
    ax.set_title(f'Protein: After arcsinh(x/cofactor)\n(per-marker cofactors)', fontsize=10)
    ax.set_xlabel('arcsinh(MFI/cofactor)')
    ax.legend(fontsize=8)
    
    ax = axes[1, 2]
    norm_vals = protein_shared_after[:, best_feat_idx]
    nonzero_mask = norm_vals > ZERO_VALUE + 0.1
    ax.hist(norm_vals[nonzero_mask], bins=30, alpha=0.7, color='forestgreen', edgecolor='white',
            label=f'Detected ({nonzero_mask.sum():,})')
    ax.axvline(x=ZERO_VALUE, color='red', linestyle='--', linewidth=2,
               label=f'Not detected ({(~nonzero_mask).sum():,})')
    ax.set_title('Protein: Detection-Aware Normalized\n(background → fixed value)', fontsize=10)
    ax.set_xlabel('Normalized value')
    ax.legend(fontsize=8)
    
    ax = axes[1, 3]
    ax.axis('off')
    ax.text(0.1, 0.85, 'Protein Pipeline:', fontsize=12, fontweight='bold', transform=ax.transAxes)
    ax.text(0.1, 0.70, '1. arcsinh(x/cofactor) per marker', fontsize=10, transform=ax.transAxes)
    ax.text(0.15, 0.58, '• Cofactor = p25 * 2 (data-driven)', fontsize=9, transform=ax.transAxes)
    ax.text(0.1, 0.46, '2. Detection via Cell Median:', fontsize=10, transform=ax.transAxes)
    ax.text(0.15, 0.34, '• Threshold = p25 * 0.5 (per marker)', fontsize=9, transform=ax.transAxes)
    ax.text(0.15, 0.22, '• Below threshold → background', fontsize=9, transform=ax.transAxes)
    
    # ============================================================
    # Row 3: Distribution comparison - NON-ZERO VALUES ONLY
    # ============================================================
    
    # Get non-zero values only
    rna_nonzero = rna_shared_after_scale[rna_shared_after_scale > ZERO_VALUE + 0.1]
    prot_nonzero = protein_shared_after[protein_shared_after > ZERO_VALUE + 0.1]
    
    ax = axes[2, 0]
    bins = np.linspace(-3, 3, 50)
    ax.hist(rna_nonzero, bins=bins, alpha=0.6, density=True, label=f'RNA ({len(rna_nonzero):,})', color='steelblue')
    ax.hist(prot_nonzero, bins=bins, alpha=0.6, density=True, label=f'Protein ({len(prot_nonzero):,})', color='darkorange')
    ax.set_title('Non-Zero Values Only\n(should overlap well)', fontsize=10)
    ax.set_xlabel('Normalized value')
    ax.set_ylabel('Density')
    ax.legend(fontsize=8)
    
    # Box plot of non-zero values
    ax = axes[2, 1]
    rna_sample = rna_nonzero[::max(1, len(rna_nonzero)//5000)]
    prot_sample = prot_nonzero[::max(1, len(prot_nonzero)//5000)]
    bp = ax.boxplot([rna_sample, prot_sample], labels=['RNA', 'Protein'], patch_artist=True)
    bp['boxes'][0].set_facecolor('steelblue')
    bp['boxes'][1].set_facecolor('darkorange')
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax.set_ylabel('Normalized value (non-zero only)')
    ax.set_title('Overall Distribution (Non-Zero Values)', fontsize=10)
    
    # Per-feature mean of NON-ZERO values (should be ~0)
    ax = axes[2, 2]
    rna_means_nz = []
    prot_means_nz = []
    for i in range(n_features):
        rna_nz = rna_shared_after_scale[:, i][rna_shared_after_scale[:, i] > ZERO_VALUE + 0.1]
        prot_nz = protein_shared_after[:, i][protein_shared_after[:, i] > ZERO_VALUE + 0.1]
        rna_means_nz.append(rna_nz.mean() if len(rna_nz) > 0 else 0)
        prot_means_nz.append(prot_nz.mean() if len(prot_nz) > 0 else 0)
    
    ax.bar(x_pos - width/2, rna_means_nz, width, label='RNA', alpha=0.8, color='steelblue')
    ax.bar(x_pos + width/2, prot_means_nz, width, label='Protein', alpha=0.8, color='darkorange')
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax.set_xlabel('Feature index')
    ax.set_ylabel('Mean (non-zero only)')
    ax.set_title('Per-Feature Mean (Non-Zero Values)', fontsize=10)
    ax.legend(fontsize=8)
    ax.set_xticks(x_pos)
    
    # Per-feature std of NON-ZERO values (should be ~1)
    ax = axes[2, 3]
    rna_stds_nz = []
    prot_stds_nz = []
    for i in range(n_features):
        rna_nz = rna_shared_after_scale[:, i][rna_shared_after_scale[:, i] > ZERO_VALUE + 0.1]
        prot_nz = protein_shared_after[:, i][protein_shared_after[:, i] > ZERO_VALUE + 0.1]
        rna_stds_nz.append(rna_nz.std() if len(rna_nz) > 1 else 0)
        prot_stds_nz.append(prot_nz.std() if len(prot_nz) > 1 else 0)
    
    ax.bar(x_pos - width/2, rna_stds_nz, width, label='RNA', alpha=0.8, color='steelblue')
    ax.bar(x_pos + width/2, prot_stds_nz, width, label='Protein', alpha=0.8, color='darkorange')
    ax.axhline(y=1, color='red', linestyle='--', alpha=0.5)
    ax.set_xlabel('Feature index')
    ax.set_ylabel('Std Dev (non-zero only)')
    ax.set_title('Per-Feature Std Dev (Non-Zero Values)', fontsize=10)
    ax.legend(fontsize=8)
    ax.set_xticks(x_pos)
    
    # ============================================================
    # Row 4: Detection rates and summary
    # ============================================================
    
    # Detection rate comparison
    ax = axes[3, 0]
    ax.bar(x_pos - width/2, rna_det_rates, width, label='RNA', alpha=0.8, color='steelblue')
    ax.bar(x_pos + width/2, prot_det_rates, width, label='Protein', alpha=0.8, color='darkorange')
    ax.set_xlabel('Feature index')
    ax.set_ylabel('Detection rate (%)')
    ax.set_title('Per-Feature Detection Rate', fontsize=10)
    ax.legend(fontsize=8)
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f[:6] for f in feature_names], rotation=45, ha='right', fontsize=8)
    
    # Detection rate scatter
    ax = axes[3, 1]
    ax.scatter(rna_det_rates, prot_det_rates, s=50, alpha=0.7)
    for i, fname in enumerate(feature_names):
        ax.annotate(fname[:6], (rna_det_rates[i], prot_det_rates[i]), fontsize=7)
    ax.plot([0, 100], [0, 100], 'r--', alpha=0.5)
    ax.set_xlabel('RNA detection rate (%)')
    ax.set_ylabel('Protein detection rate (%)')
    ax.set_title('Detection Rate Comparison', fontsize=10)
    
    # Empty placeholder
    ax = axes[3, 2]
    ax.axis('off')
    
    # Summary text
    ax = axes[3, 3]
    ax.axis('off')
    avg_threshold = np.mean(list(marker_thresholds.values()))
    summary = f"""DETECTION-AWARE NORMALIZATION SUMMARY
    {"="*45}
    
    Cell counts:
      RNA:     {rna_shared_raw.shape[0]:>10,} cells
      Protein: {protein_shared_raw.shape[0]:>10,} cells
    
    Detection method:
      RNA: count > 0
      Protein: Cell Median > per-marker threshold
      Average threshold: {avg_threshold:.1f}
    
    Non-zero values after normalization:
      RNA:     mean={np.mean(rna_means_nz):.3f}, std={np.mean(rna_stds_nz):.3f}
      Protein: mean={np.mean(prot_means_nz):.3f}, std={np.mean(prot_stds_nz):.3f}
    
    Zero handling:
      All "not detected" set to: {ZERO_VALUE}
      (Below all detected values)
    """
    ax.text(0.0, 0.95, summary, transform=ax.transAxes, fontsize=9,
            verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.suptitle('Detection-Aware Normalization Results', fontsize=14, y=1.01)
    plt.show()
    
    print("\n" + "="*70)
    print("KEY INSIGHT:")
    print("="*70)
    print("RNA detection: count > 0")
    print(f"Protein detection: Cell Median > per-marker threshold (avg: {avg_threshold:.1f})")
    print("This ensures 'not detected' is biologically meaningful in both modalities.")


In [ ]:
# Convert to numpy arrays
rna_shared = rna_shared_adata.X.copy()
if sparse.issparse(rna_shared):
    rna_shared = rna_shared.toarray()
    
protein_shared = protein_shared_adata.X.copy()
if sparse.issparse(protein_shared):
    protein_shared = protein_shared.toarray()

# Remove zero-variance features
rna_std = rna_shared.std(axis=0)
prot_std = protein_shared.std(axis=0)
valid_mask = (rna_std > 1e-6) & (prot_std > 1e-6)

if not valid_mask.all():
    print(f"Removing {(~valid_mask).sum()} zero-variance features")
    rna_shared = rna_shared[:, valid_mask]
    protein_shared = protein_shared[:, valid_mask]
    # Update correspondence
    rna_protein_correspondence = rna_protein_correspondence[valid_mask]

print(f"\nFinal shared arrays:")
print(f"  rna_shared: {rna_shared.shape}")
print(f"  protein_shared: {protein_shared.shape}")

In [ ]:
# Analyze shared feature quality
print("=" * 60)
print("SHARED FEATURE QUALITY ANALYSIS")
print("=" * 60)

# Get raw counts for analysis (before normalization)
rna_raw_shared = rna_adata[:, rna_protein_correspondence[:, 0]].X
if sparse.issparse(rna_raw_shared):
    rna_raw_shared = rna_raw_shared.toarray()

protein_raw_shared = protein_adata[:, rna_protein_correspondence[:, 1]].X
if sparse.issparse(protein_raw_shared):
    protein_raw_shared = protein_raw_shared.toarray()

# Calculate statistics for each feature
feature_stats = []
for i, (rna_gene, prot_marker) in enumerate(rna_protein_correspondence):
    rna_col = rna_raw_shared[:, i]
    prot_col = protein_raw_shared[:, i]
    
    # % cells expressing
    rna_pct_expressing = (rna_col > 0).sum() / len(rna_col) * 100
    prot_pct_expressing = (prot_col > 0).sum() / len(prot_col) * 100
    
    feature_stats.append({
        'RNA name': rna_gene,
        'Protein': prot_marker,
        'RNA_%_expressing': rna_pct_expressing,
        'Prot_%_expressing': prot_pct_expressing,
    })

stats_df = pd.DataFrame(feature_stats)
stats_df = stats_df.sort_values('RNA_%_expressing', ascending=True)

print("\nFeature-by-feature statistics (sorted by RNA detection rate):")
print(stats_df.to_string(index=False))

# Summary
print(f"\n{'='*60}")
print("SUMMARY:")
avg_rna_detection = stats_df['RNA_%_expressing'].mean()
avg_prot_detection = stats_df['Prot_%_expressing'].mean()
print(f"  Average RNA detection rate: {avg_rna_detection:.1f}% of cells")
print(f"  Average Protein detection rate: {avg_prot_detection:.1f}% of cells")

# Warning for sparse features
rare_features = stats_df[stats_df['RNA_%_expressing'] < 10]
if len(rare_features) > 0:
    print(f"\n  NOTE: {len(rare_features)} features detected in <10% of RNA cells")
    print("  These provide weaker signal for matching.")

In [ ]:
# Protein active - use all markers (except DAPI)
protein_markers_active = [m for m in protein_adata.var_names if m != 'DAPI']
protein_adata_active = protein_adata[:, protein_markers_active].copy()

# Scale if needed
prot_mean = protein_adata_active.X.mean()
if abs(prot_mean) > 0.1:
    sc.pp.scale(protein_adata_active)
    
print(f"Protein active: {protein_adata_active.shape}")

In [ ]:
# Extract numpy arrays
rna_active = rna_adata.X.copy()
if sparse.issparse(rna_active):
    rna_active = rna_active.toarray()

protein_active = protein_adata_active.X.copy()
if sparse.issparse(protein_active):
    protein_active = protein_active.toarray()

# Remove zero-variance features
rna_active = rna_active[:, rna_active.std(axis=0) > 1e-6]
protein_active = protein_active[:, protein_active.std(axis=0) > 1e-6]

print(f"\nFinal active arrays:")
print(f"  rna_active (HVGs): {rna_active.shape}")
print(f"  protein_active: {protein_active.shape}")

In [ ]:
# CRITICAL VALIDATION: Check array dimensions match
print("=" * 50)
print("DIMENSION VALIDATION")
print("=" * 50)
print(f"RNA shared cells:     {rna_shared.shape[0]}")
print(f"RNA active cells:     {rna_active.shape[0]}")
print(f"Protein shared cells: {protein_shared.shape[0]}")
print(f"Protein active cells: {protein_active.shape[0]}")
print()

assert rna_shared.shape[0] == rna_active.shape[0], \
    f"RNA mismatch: shared={rna_shared.shape[0]}, active={rna_active.shape[0]}"
assert protein_shared.shape[0] == protein_active.shape[0], \
    f"Protein mismatch: shared={protein_shared.shape[0]}, active={protein_active.shape[0]}"
assert rna_shared.shape[1] == protein_shared.shape[1], \
    f"Shared feature mismatch: RNA={rna_shared.shape[1]}, Protein={protein_shared.shape[1]}"

print("All dimensions validated!")
print(f"\nIntegrating {rna_active.shape[0]} RNA cells with {protein_active.shape[0]} protein cells")
print(f"Using {rna_shared.shape[1]} shared features for initialization")

In [ ]:
# Save checkpoint for cross-modal integration
# This allows skipping the MARIO section and going directly to MaxFuse

checkpoint_dir = 'results/2_integration'
os.makedirs(checkpoint_dir, exist_ok=True)

# Save arrays needed for MaxFuse
np.save(f'{checkpoint_dir}/checkpoint_rna_shared.npy', rna_shared)
np.save(f'{checkpoint_dir}/checkpoint_protein_shared.npy', protein_shared)
np.save(f'{checkpoint_dir}/checkpoint_rna_active.npy', rna_active)
np.save(f'{checkpoint_dir}/checkpoint_protein_active.npy', protein_active)

# Save correspondence
correspondence_checkpoint = pd.DataFrame(
    rna_protein_correspondence, 
    columns=['rna_gene', 'protein_marker']
)
correspondence_checkpoint.to_csv(f'{checkpoint_dir}/checkpoint_correspondence.csv', index=False)

print(f'Checkpoint saved to {checkpoint_dir}/')
print(f'  - checkpoint_rna_shared.npy: {rna_shared.shape}')
print(f'  - checkpoint_protein_shared.npy: {protein_shared.shape}')
print(f'  - checkpoint_rna_active.npy: {rna_active.shape}')
print(f'  - checkpoint_protein_active.npy: {protein_active.shape}')
print(f'  - checkpoint_correspondence.csv')
print(f'\nYou can now skip to MaxFuse Integration (Step 7) if desired.')

---
# MARIO Integration (Optional - Same-Modality Data Only)

**IMPORTANT**: MARIO is designed for **same-modality integration** (e.g., protein-protein).

## When to Use MARIO
- Both datasets measure the **same biological quantity** (e.g., CITE-seq vs CyTOF)
- Shared features are the **same measurements** by different technologies
- Values should be **correlated but not identical** between modalities

## When to Skip MARIO
- **Cross-modal integration** (e.g., RNA-seq vs protein/CODEX) → **Skip to MaxFuse**
- Shared features represent **different biological measurements** (mRNA vs protein)
- Canonical correlations are all ~1.0 (indicates data incompatibility)

## Current Data Type
This notebook integrates **RNA-seq with CODEX protein data** - a cross-modal scenario.

**Recommendation**: Skip this section and proceed to **MaxFuse Integration (Step 7)**.

---

If you still want to run MARIO (e.g., for comparison or same-modality data), continue below.

## Step 5: MARIO - Matchability Test (Pre-Integration Diagnostic)

Before running integration, we test whether the two datasets have meaningful correspondence.
MARIO uses random sign flips to create a null distribution and computes p-values.

- **Low p-value** (< 0.05): Datasets are matchable
- **High p-value** (> 0.05): No significant correspondence detected

### Data Validation

Check for and handle NaN/Inf values that may result from normalization of sparse features.

In [ ]:
# Check and handle NaN/Inf values before MARIO
# Detection-aware normalization can produce NaN for problematic features

print("Checking for NaN/Inf values in shared arrays...")
print(f"  rna_shared: NaN={np.isnan(rna_shared).sum()}, Inf={np.isinf(rna_shared).sum()}")
print(f"  protein_shared: NaN={np.isnan(protein_shared).sum()}, Inf={np.isinf(protein_shared).sum()}")

# Replace NaN/Inf with 0 (these are likely failed normalizations for sparse features)
if np.isnan(rna_shared).any() or np.isinf(rna_shared).any():
    print("\nCleaning rna_shared...")
    rna_shared = np.nan_to_num(rna_shared, nan=0.0, posinf=0.0, neginf=0.0)
    
if np.isnan(protein_shared).any() or np.isinf(protein_shared).any():
    print("Cleaning protein_shared...")
    protein_shared = np.nan_to_num(protein_shared, nan=0.0, posinf=0.0, neginf=0.0)

# Also check active arrays
print(f"\n  rna_active: NaN={np.isnan(rna_active).sum()}, Inf={np.isinf(rna_active).sum()}")
print(f"  protein_active: NaN={np.isnan(protein_active).sum()}, Inf={np.isinf(protein_active).sum()}")

if np.isnan(rna_active).any() or np.isinf(rna_active).any():
    print("\nCleaning rna_active...")
    rna_active = np.nan_to_num(rna_active, nan=0.0, posinf=0.0, neginf=0.0)
    
if np.isnan(protein_active).any() or np.isinf(protein_active).any():
    print("Cleaning protein_active...")
    protein_active = np.nan_to_num(protein_active, nan=0.0, posinf=0.0, neginf=0.0)

print("\nArrays cleaned and ready for integration.")

In [ ]:
# Subsample for MARIO (MARIO requires n1 <= n2, and for speed we subsample)
np.random.seed(42)

# MARIO needs RNA (smaller) to be df1 and Protein (larger) to be df2
n_rna_subsample = min(2000, rna_shared.shape[0])
n_prot_subsample = min(10000, protein_shared.shape[0])

rna_idx_subsample = np.random.choice(rna_shared.shape[0], n_rna_subsample, replace=False)
prot_idx_subsample = np.random.choice(protein_shared.shape[0], n_prot_subsample, replace=False)

# Create DataFrames with overlapping column names (required by MARIO)
shared_feature_names = [f"feat_{i}" for i in range(rna_shared.shape[1])]

# Extract subsamples and ensure no NaN values
rna_subsample = rna_shared[rna_idx_subsample].copy()
prot_subsample = protein_shared[prot_idx_subsample].copy()

# Final NaN check on subsamples
rna_subsample = np.nan_to_num(rna_subsample, nan=0.0, posinf=0.0, neginf=0.0)
prot_subsample = np.nan_to_num(prot_subsample, nan=0.0, posinf=0.0, neginf=0.0)

rna_df_mario = pd.DataFrame(rna_subsample, columns=shared_feature_names)
prot_df_mario = pd.DataFrame(prot_subsample, columns=shared_feature_names)

print(f"MARIO subsample sizes:")
print(f"  RNA: {rna_df_mario.shape}")
print(f"  Protein: {prot_df_mario.shape}")
print(f"  NaN in RNA df: {rna_df_mario.isna().sum().sum()}")
print(f"  NaN in Protein df: {prot_df_mario.isna().sum().sum()}")

In [ ]:
# Initialize MARIO
mario = Mario(rna_df_mario, prot_df_mario, normalization=True)

# Specify matching parameters
# n_matched_per_cell: how many protein cells to match with each RNA cell
n_matched = max(1, n_prot_subsample // n_rna_subsample)
mario.specify_matching_params(n_matched_per_cell=n_matched)

print(f"Matching {n_matched} protein cells per RNA cell")

In [ ]:
# Compute distance using overlapping features
n_ovlp_components = min(15, rna_shared.shape[1] - 1)
dist_ovlp, singular_vals = mario.compute_dist_ovlp(n_components=n_ovlp_components)

print(f"Distance matrix shape: {dist_ovlp.shape}")
print(f"Singular values: {singular_vals[:5]}...")

# Plot singular values
plt.figure(figsize=(8, 4))
plt.plot(singular_vals, 'bo-')
plt.xlabel('Component')
plt.ylabel('Singular Value')
plt.title('MARIO: Singular Values of Stacked Overlap Features')
plt.show()

In [ ]:
# Initial matching using overlap features
print("Finding initial matching using overlap features...")
matching_ovlp = mario.match_cells('ovlp', sparsity=None, mode='auto')

# Count matched cells
n_matched_cells = sum(1 for m in matching_ovlp if len(m) > 0)
print(f"Matched {n_matched_cells}/{len(matching_ovlp)} RNA cells")

In [ ]:
# Add active features (all HVGs) for refined matching
# For MARIO, we need DataFrames with:
# - Overlapping columns (shared features) with same names
# - Non-overlapping columns (active features) with different names

# RNA: shared features + active features
rna_active_subsample = rna_active[rna_idx_subsample].copy()
rna_active_subsample = np.nan_to_num(rna_active_subsample, nan=0.0, posinf=0.0, neginf=0.0)
rna_active_names = [f"rna_feat_{i}" for i in range(rna_active_subsample.shape[1])]

rna_df_full = pd.DataFrame(
    np.hstack([rna_subsample, rna_active_subsample]),
    columns=shared_feature_names + rna_active_names
)

# Protein: shared features + active features
prot_active_subsample = protein_active[prot_idx_subsample].copy()
prot_active_subsample = np.nan_to_num(prot_active_subsample, nan=0.0, posinf=0.0, neginf=0.0)
prot_active_names = [f"prot_feat_{i}" for i in range(prot_active_subsample.shape[1])]

prot_df_full = pd.DataFrame(
    np.hstack([prot_subsample, prot_active_subsample]),
    columns=shared_feature_names + prot_active_names
)

print(f"Full DataFrames for MARIO:")
print(f"  RNA: {rna_df_full.shape} ({len(shared_feature_names)} shared + {len(rna_active_names)} active)")
print(f"  Protein: {prot_df_full.shape} ({len(shared_feature_names)} shared + {len(prot_active_names)} active)")
print(f"  NaN check - RNA: {rna_df_full.isna().sum().sum()}, Protein: {prot_df_full.isna().sum().sum()}")

In [ ]:
# Create new MARIO object with full features
mario_full = Mario(rna_df_full, prot_df_full, normalization=False)
mario_full.specify_matching_params(n_matched_per_cell=n_matched)

# Compute distance using overlap features
_ = mario_full.compute_dist_ovlp(n_components=n_ovlp_components)

# Initial matching
_ = mario_full.match_cells('ovlp', sparsity=None, mode='auto')

# Compute distance using ALL features (CCA refinement)
# NOTE: Use conservative number of CCA components.
# With few protein features relative to RNA features and matched samples,
# CCA can find trivially perfect correlations if given too many components.
# Rule of thumb: use min(n_shared - 1, sqrt(n_prot_active))
n_prot_active = prot_df_full.shape[1] - len(shared_feature_names)
n_cca_components = min(
    len(shared_feature_names) - 1,  # No more than shared features
    int(np.sqrt(n_prot_active)) + 1,  # Conservative based on protein features
    8  # Hard cap for this data
)
n_cca_components = max(3, n_cca_components)  # At least 3 components
print(f"Using {n_cca_components} CCA components")
print(f"  (shared features: {len(shared_feature_names)}, protein active: {n_prot_active})")

dist_all, cancor = mario_full.compute_dist_all('ovlp', n_components=n_cca_components)

# Interpret canonical correlations
print(f"\nCanonical correlations: {np.round(cancor, 4)}")

if np.allclose(cancor, 1.0, atol=0.01):
    print("\nNOTE: Canonical correlations are very high (~1.0).")
    print("This is common when protein features are few relative to matched samples.")
    print("The CCA can perfectly align matched pairs in this low-dimensional space.")
    print("Matching quality depends on how well CCA generalizes to unmatched cells.")
elif np.mean(cancor) > 0.7:
    print("\nGood: High canonical correlations indicate strong alignment.")
else:
    print("\nNote: Moderate correlations - may indicate weaker cross-modal alignment.")

# Plot canonical correlations
plt.figure(figsize=(8, 4))
plt.bar(range(len(cancor)), cancor)
plt.xlabel('CCA Component')
plt.ylabel('Canonical Correlation')
plt.title('MARIO: Canonical Correlations')
plt.ylim(0, 1.1)
plt.axhline(y=0.7, color="orange", linestyle="--", alpha=0.5, label="Good threshold")
plt.legend()
plt.show()

In [ ]:
# Match using all features
matching_all = mario_full.match_cells('all', sparsity=None, mode='auto')

n_matched_all = sum(1 for m in matching_all if len(m) > 0)
print(f"Matched {n_matched_all}/{len(matching_all)} RNA cells using all features")

In [ ]:
# DIAGNOSTIC: Check canonical correlations before matchability test

print("="*60)
print("MATCHABILITY DIAGNOSTIC")
print("="*60)

# Check the canonical correlations from the existing matching
print("\n1. OBSERVED CANONICAL CORRELATIONS:")
print(f"   From compute_dist_all (stored): {cancor[:5] if 'cancor' in dir() else 'Not computed'}")

# Check data properties
print("\n2. DATA PROPERTIES:")
print(f"   mario_full.df1 shape: {mario_full.df1.shape}")
print(f"   mario_full.df2 shape: {mario_full.df2.shape}")
print(f"   Overlap features: {len(mario_full.ovlp_features)}")

# Check for zero-variance features
df1_std = mario_full.df1.std()
df2_std = mario_full.df2.std()
print(f"\n   df1 zero-variance features: {(df1_std < 1e-10).sum()}")
print(f"   df2 zero-variance features: {(df2_std < 1e-10).sum()}")

# Check data scale
print("\n3. DATA SCALE:")
print(f"   df1 mean: {mario_full.df1.values.mean():.4f}, std: {mario_full.df1.values.std():.4f}")
print(f"   df2 mean: {mario_full.df2.values.mean():.4f}, std: {mario_full.df2.values.std():.4f}")

# CHECK MATCHING - this is critical
print("\n4. MATCHING STATISTICS:")
n_matched_ovlp = sum(1 for m in mario_full.matching['ovlp'] if len(m) > 0)
n_matched_all = sum(1 for m in mario_full.matching['all'] if len(m) > 0)
print(f"   Cells matched (overlap): {n_matched_ovlp} / {mario_full.n1}")
print(f"   Cells matched (all):     {n_matched_all} / {mario_full.n1}")

# Check the aligned data dimensions for CCA
from mario import embed
X_aligned = []
Y_aligned = []
for ii in range(mario_full.n1):
    if len(mario_full.matching['ovlp'][ii]) > 0:
        X_aligned.append(mario_full.df1.iloc[ii, :].values)
        Y_aligned.append(mario_full.df2.iloc[mario_full.matching['ovlp'][ii]].mean(axis=0).values)

X_aligned = np.array(X_aligned)
Y_aligned = np.array(Y_aligned)
print(f"\n5. CCA INPUT DIMENSIONS:")
print(f"   X (RNA) aligned: {X_aligned.shape}")
print(f"   Y (Protein) aligned: {Y_aligned.shape}")
print(f"   Ratio features/samples (RNA): {X_aligned.shape[1]/X_aligned.shape[0]:.1f}")

# THE PROBLEM: CCA with features >> samples gives trivial perfect correlations!
if X_aligned.shape[1] > X_aligned.shape[0]:
    print("\n   ⚠️  WARNING: More features than samples!")
    print("   CCA will overfit and give meaningless correlations of 1.0")
    print("   This is why matchability test returns p=1")

# Test CCA with ONLY overlap features
print("\n6. CCA WITH OVERLAP FEATURES ONLY:")
X_ovlp = mario_full.df1[mario_full.ovlp_features].iloc[[i for i in range(mario_full.n1) if len(mario_full.matching['ovlp'][i]) > 0]].values
Y_ovlp = np.array([mario_full.df2[mario_full.ovlp_features].iloc[mario_full.matching['ovlp'][i]].mean(axis=0).values 
                   for i in range(mario_full.n1) if len(mario_full.matching['ovlp'][i]) > 0])
print(f"   X_ovlp shape: {X_ovlp.shape}")
print(f"   Y_ovlp shape: {Y_ovlp.shape}")

try:
    n_comp = min(5, X_ovlp.shape[1]-1, X_ovlp.shape[0]-1)
    cancor_ovlp_only, _ = embed.get_cancor(X_ovlp, Y_ovlp, n_components=n_comp)
    print(f"   Canonical correlations (overlap only): {cancor_ovlp_only}")
    print(f"   Mean: {np.mean(cancor_ovlp_only):.4f}")
except Exception as e:
    print(f"   Error: {e}")

print("\n" + "="*60)


In [ ]:
# Run matchability test
print("=" * 60)
print("MARIO MATCHABILITY TEST")
print("=" * 60)
print("\nRunning statistical test for dataset matchability...")
print("(This uses random sign flips to create null distribution)")
print()

# CRITICAL FIX: Clean NaN/Inf values in MARIO dataframes before matchability test
# The matchability test internally uses CCA which cannot handle NaN values
print("Cleaning MARIO dataframes for CCA compatibility...")

# Clean df1 (RNA) - aggressive column-by-column approach
mario_full.df1 = mario_full.df1.copy()
for col in mario_full.df1.columns:
    mario_full.df1[col] = np.nan_to_num(mario_full.df1[col].values, nan=0.0, posinf=0.0, neginf=0.0)

# Clean df2 (Protein) - aggressive column-by-column approach
mario_full.df2 = mario_full.df2.copy()
for col in mario_full.df2.columns:
    mario_full.df2[col] = np.nan_to_num(mario_full.df2[col].values, nan=0.0, posinf=0.0, neginf=0.0)

# Verify no NaN/Inf remain
df1_clean = not (np.isnan(mario_full.df1.values).any() or np.isinf(mario_full.df1.values).any())
df2_clean = not (np.isnan(mario_full.df2.values).any() or np.isinf(mario_full.df2.values).any())
print(f"  df1 clean: {df1_clean} (NaN: {np.isnan(mario_full.df1.values).sum()}, Inf: {np.isinf(mario_full.df1.values).sum()})")
print(f"  df2 clean: {df2_clean} (NaN: {np.isnan(mario_full.df2.values).sum()}, Inf: {np.isinf(mario_full.df2.values).sum()})")
assert df1_clean and df2_clean, "Failed to clean NaN/Inf values"

# Ensure both initial (ovlp) and refined (all) matching are complete
print("Verifying initial and refined matching are complete...")

# Check if matching has been done, if not redo it
if not hasattr(mario_full, 'matching_ovlp') or mario_full.matching_ovlp is None:
    print("  Re-running initial matching (overlap features)...")
    mario_full.match_cells('ovlp', sparsity=None, mode='auto')

if not hasattr(mario_full, 'matching_all') or mario_full.matching_all is None:
    print("  Re-running refined matching (all features)...")
    mario_full.match_cells('all', sparsity=None, mode='auto')

print("  Both matchings confirmed. Proceeding with matchability test...\n")

# Note: This can take a few minutes
# Reduce n_sim if it takes too long
pval_ovlp, pval_all = mario_full.matchable(
    n_sim=10,           # Number of simulations (increase for more accuracy)
    top_k=5,            # Use top-k canonical correlations
    flip_prob=0.3,      # Probability of sign flip
    subsample_prop=1,   # Subsample for speed
    verbose=True
)

print(f"\n{'='*60}")
print("MATCHABILITY TEST RESULTS")
print("="*60)
print(f"P-value (overlap features only): {pval_ovlp:.4f}")
print(f"P-value (all features):          {pval_all:.4f}")
print()

if pval_ovlp < 0.05 or pval_all < 0.05:
    print("RESULT: Datasets appear to be MATCHABLE (p < 0.05)")
    print("  The correspondence between modalities is statistically significant.")
else:
    print("RESULT: Datasets may NOT be well-matched (p >= 0.05)")
    print("  Proceed with caution - results may be unreliable.")



## Step 6: MARIO - Interpolation (Optimal Weight Search)

MARIO searches for the optimal weight between:
- Distance from **overlap features only**
- Distance from **all features** (via CCA)

The optimal weight is selected based on canonical correlations.

In [ ]:
# Run interpolation to find optimal weight
print("Searching for optimal interpolation weight...")
print("(Testing weights from 0 to 1)")
print()

best_wt, best_matching = mario_full.interpolate(
    n_wts=10,     # Number of weights to try
    top_k=5,      # Use top-k canonical correlations to evaluate
    verbose=True
)

print(f"\nOptimal weight: {best_wt:.2f}")
print(f"  (0 = use only overlap features, 1 = use only CCA features)")

n_matched_best = sum(1 for m in best_matching if len(m) > 0)
print(f"\nMatched {n_matched_best}/{len(best_matching)} RNA cells with optimal weight")

In [ ]:
# Filter bad matches using joint regularized clustering
print("\nFiltering bad matches using joint regularized clustering...")

n_clusters_filter = min(15, n_rna_subsample // 50)  # Aim for ~50 cells per cluster
n_clusters_filter = max(5, n_clusters_filter)

filtered_matching = mario_full.filter_bad_matches(
    matching='wted',           # Use the interpolated matching
    n_clusters=n_clusters_filter,
    n_components=min(15, n_cca_components),
    bad_prop=0.1,              # Remove ~10% of worst matches
    max_iter=30,
    verbose=True
)

n_matched_filtered = sum(1 for m in filtered_matching if len(m) > 0)
print(f"\nAfter filtering: {n_matched_filtered}/{len(filtered_matching)} RNA cells matched")
print(f"Removed {n_matched_best - n_matched_filtered} bad matches")

In [ ]:
# Optional: KNN matching for softer assignments
knn_matching = mario_full.knn_matching(dist_mat='wted', k=5)

print(f"KNN matching: each RNA cell matched to {5} nearest protein cells")

In [ ]:
# Compute CCA embedding for visualization
from mario import embed

# Align the datasets using the filtered matching
X_aligned = []
Y_aligned = []
matched_rna_indices_mario = []
matched_prot_indices_mario = []

for i, matches in enumerate(filtered_matching):
    if len(matches) > 0:
        X_aligned.append(rna_df_full.iloc[i].values)
        # Average the matched protein cells
        Y_aligned.append(prot_df_full.iloc[matches].mean(axis=0).values)
        matched_rna_indices_mario.append(rna_idx_subsample[i])
        matched_prot_indices_mario.append(prot_idx_subsample[matches[0]])  # Take first match

X_aligned = np.array(X_aligned)
Y_aligned = np.array(Y_aligned)

print(f"Aligned arrays: RNA {X_aligned.shape}, Protein {Y_aligned.shape}")

# Fit CCA for embedding
embed_dim = min(20, X_aligned.shape[1], Y_aligned.shape[1])
cancor_embed, cca = embed.get_cancor(X_aligned, Y_aligned, n_components=embed_dim)

# Get CCA scores
rna_cca_mario, prot_cca_mario = cca.transform(X_aligned, Y_aligned)

print(f"MARIO CCA embedding: {rna_cca_mario.shape}")

In [ ]:
# Visualize MARIO results
from sklearn.manifold import TSNE

# Combine embeddings
combined_mario = np.vstack([rna_cca_mario, prot_cca_mario])
labels_mario = ['RNA'] * len(rna_cca_mario) + ['Protein'] * len(prot_cca_mario)

# Run t-SNE (faster than UMAP for small datasets)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
embedding_2d = tsne.fit_transform(combined_mario[:, :10])  # Use first 10 CCA components

# Plot
plt.figure(figsize=(10, 8))
for label in ['RNA', 'Protein']:
    mask = np.array(labels_mario) == label
    plt.scatter(embedding_2d[mask, 0], embedding_2d[mask, 1], 
                label=label, alpha=0.5, s=10)
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title('MARIO: Joint Embedding (t-SNE of CCA scores)')
plt.legend()
plt.show()

---
# MaxFuse Integration (Recommended for Cross-Modal Data)

MaxFuse is specifically designed for **cross-modal integration** with:
- **Weak linkage**: Few or uninformative shared features
- **Different modalities**: RNA-seq, CODEX/protein, ATAC-seq, etc.
- **Scalable architecture**: Batch processing with pivot propagation

## Key Features
1. **Graph-based smoothing**: Reduces noise before matching
2. **Iterative CCA refinement**: Improves alignment quality
3. **Pivot propagation**: Scales to large datasets

---

In [ ]:
# Load checkpoint data if MARIO section was skipped
# This cell is idempotent - safe to run even if data already loaded

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from maxfuse import Fusor

checkpoint_dir = 'results/2_integration'

# Check if we need to load from checkpoint
need_checkpoint = False
try:
    # Check if required variables exist and have data
    _ = rna_shared.shape
    _ = protein_shared.shape
    _ = rna_active.shape
    _ = protein_active.shape
    print('Data already loaded - using existing arrays.')
except NameError:
    need_checkpoint = True
    print('Data not found - loading from checkpoint...')

if need_checkpoint:
    # Load arrays from checkpoint
    rna_shared = np.load(f'{checkpoint_dir}/checkpoint_rna_shared.npy')
    protein_shared = np.load(f'{checkpoint_dir}/checkpoint_protein_shared.npy')
    rna_active = np.load(f'{checkpoint_dir}/checkpoint_rna_active.npy')
    protein_active = np.load(f'{checkpoint_dir}/checkpoint_protein_active.npy')
    
    # Load correspondence
    correspondence_df = pd.read_csv(f'{checkpoint_dir}/checkpoint_correspondence.csv')
    rna_protein_correspondence = correspondence_df.values
    
    print(f'Loaded from checkpoint:')
    print(f'  rna_shared: {rna_shared.shape}')
    print(f'  protein_shared: {protein_shared.shape}')
    print(f'  rna_active: {rna_active.shape}')
    print(f'  protein_active: {protein_active.shape}')


## Step 7: MaxFuse Integration

MaxFuse performs the integration in several stages:
1. Split data into batches for scalability
2. Construct k-NN graphs and cluster cells
3. Find initial pivot matches using shared features
4. Refine pivots using CCA on all features
5. Propagate matching to all cells

In [ ]:
# Create Fusor - let MaxFuse cluster automatically
fusor = Fusor(
    shared_arr1=rna_shared,
    shared_arr2=protein_shared,
    active_arr1=rna_active,
    active_arr2=protein_active,
    labels1=None,  # Let MaxFuse cluster
    labels2=None,
    method='centroid_shrinkage'
)

In [ ]:
# Calculate appropriate batching parameters
n_rna = rna_active.shape[0]
n_prot = protein_active.shape[0]
ratio = n_prot / n_rna

print(f"RNA cells: {n_rna}")
print(f"Protein cells: {n_prot}")
print(f"Ratio (protein/RNA): {ratio:.1f}")

# Batching parameters
max_outward = min(8000, n_rna)
matching_ratio = max(10, int(ratio) + 5)  # Adjusted for data ratio
metacell_sz = 2  # Metacell aggregation helps with noise

print(f"\nBatching parameters:")
print(f"  max_outward_size: {max_outward}")
print(f"  matching_ratio: {matching_ratio}")
print(f"  metacell_size: {metacell_sz}")

In [ ]:
fusor.split_into_batches(
    max_outward_size=max_outward,
    matching_ratio=matching_ratio,
    metacell_size=metacell_sz,
    verbose=True
)

In [ ]:
# Plot singular values to determine SVD components
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

plt.sca(axes[0])
fusor.plot_singular_values(target='active_arr1', n_components=50)
axes[0].set_title('RNA Active - Singular Values')

plt.sca(axes[1])
fusor.plot_singular_values(target='active_arr2', n_components=min(23, protein_active.shape[1]-1))
axes[1].set_title('Protein Active - Singular Values')

plt.tight_layout()
plt.show()

In [ ]:
# Set SVD components based on data dimensions
n_prot_features = protein_active.shape[1]
n_rna_features = rna_active.shape[1]
n_shared = rna_shared.shape[1]

svd_comp1_graph = min(40, n_rna_features - 1)
svd_comp2_graph = min(15, n_prot_features - 1)

print(f"Graph construction SVD components:")
print(f"  RNA: {svd_comp1_graph}")
print(f"  Protein: {svd_comp2_graph}")

In [ ]:
# Construct graphs with automatic clustering
fusor.construct_graphs(
    n_neighbors1=15,
    n_neighbors2=15,
    svd_components1=svd_comp1_graph,
    svd_components2=svd_comp2_graph,
    resolution1=2.0,   # Higher resolution = more clusters = finer smoothing
    resolution2=2.0,
    resolution_tol=0.1,
    leiden_runs=1,
    verbose=True
)

In [ ]:
# Visualize clustering results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Get cluster labels from first batch
labels1_b0 = fusor._labels1[0]
labels2_b0 = fusor._labels2[0]

# RNA cluster sizes
ax = axes[0]
unique, counts = np.unique(labels1_b0, return_counts=True)
ax.bar(range(len(counts)), sorted(counts, reverse=True))
ax.set_xlabel('Cluster rank')
ax.set_ylabel('Cells')
ax.set_title(f'RNA Clusters (n={len(unique)})')
ax.axhline(y=np.mean(counts), color='r', linestyle='--', label=f'Mean: {np.mean(counts):.0f}')
ax.legend()

# Protein cluster sizes
ax = axes[1]
unique, counts = np.unique(labels2_b0, return_counts=True)
ax.bar(range(len(counts)), sorted(counts, reverse=True))
ax.set_xlabel('Cluster rank')
ax.set_ylabel('Cells')
ax.set_title(f'Protein Clusters (n={len(unique)})')
ax.axhline(y=np.mean(counts), color='r', linestyle='--', label=f'Mean: {np.mean(counts):.0f}')
ax.legend()

# Summary stats
ax = axes[2]
ax.axis('off')
stats_text = f'''Graph Construction Summary
{"="*40}

RNA (Batch 0):
  Clusters: {len(np.unique(labels1_b0))}
  Cells: {len(labels1_b0)}
  Mean cluster size: {np.mean(np.bincount(labels1_b0)):.1f}

Protein (Batch 0):
  Clusters: {len(np.unique(labels2_b0))}
  Cells: {len(labels2_b0)}
  Mean cluster size: {np.mean(np.bincount(labels2_b0)):.1f}
'''
ax.text(0.1, 0.9, stats_text, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

In [ ]:
# Find initial pivots with smoothing for weak linkage
svd_shared1 = min(25, n_shared - 1)
svd_shared2 = min(20, n_shared - 1)
print(f"Using {svd_shared1}/{svd_shared2} SVD components for shared features")

fusor.find_initial_pivots(
    wt1=0.3,  # Smoothing weight
    wt2=0.3,
    svd_components1=svd_shared1,
    svd_components2=svd_shared2,
    verbose=True
)

In [ ]:
# Visualize initial pivot matching
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Get initial matching from first batch
init_match = fusor._init_matching[0]
init_rows, init_cols, init_scores = init_match

# Score distribution
ax = axes[0]
ax.hist(init_scores, bins=50, edgecolor='white', alpha=0.7)
ax.axvline(np.mean(init_scores), color='r', linestyle='--', 
           label=f'Mean: {np.mean(init_scores):.3f}')
ax.axvline(np.median(init_scores), color='g', linestyle='--',
           label=f'Median: {np.median(init_scores):.3f}')
ax.set_xlabel('Matching Score')
ax.set_ylabel('Count')
ax.set_title('Initial Pivot Scores')
ax.legend()

# Matches per RNA cell
ax = axes[1]
matches_per_rna = np.bincount(init_rows)
ax.hist(matches_per_rna[matches_per_rna > 0], bins=20, edgecolor='white', alpha=0.7)
ax.set_xlabel('Matches per RNA cell')
ax.set_ylabel('Count')
ax.set_title('Matching Density (RNA)')

# Summary
ax = axes[2]
ax.axis('off')
n_rna_matched = len(np.unique(init_rows))
n_prot_matched = len(np.unique(init_cols))
stats = f'''Initial Pivot Matching
{"="*40}

Total matches: {len(init_scores):,}
Unique RNA matched: {n_rna_matched:,}
Unique Protein matched: {n_prot_matched:,}

Score statistics:
  Min:    {np.min(init_scores):.4f}
  Max:    {np.max(init_scores):.4f}
  Mean:   {np.mean(init_scores):.4f}
  Median: {np.median(init_scores):.4f}
  Std:    {np.std(init_scores):.4f}
'''
ax.text(0.1, 0.9, stats, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

In [ ]:
# Check canonical correlations
cca_comp_check = min(15, n_prot_features - 1)
fusor.plot_canonical_correlations(
    svd_components1=min(30, n_rna_features - 1),
    svd_components2=None,
    cca_components=cca_comp_check
)

In [ ]:
# Refine pivots using CCA
cca_components = min(25, n_prot_features - 1)

fusor.refine_pivots(
    wt1=0.3,
    wt2=0.3,
    svd_components1=min(40, n_rna_features - 1),
    svd_components2=None,  # Keep all protein features
    cca_components=cca_components,
    n_iters=1,
    filter_prop=0.0,
    verbose=True
)

In [ ]:
# Compare initial vs refined matching
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Get refined matching
refined_match = fusor._refined_matching[0]
ref_rows, ref_cols, ref_scores = refined_match

# Score comparison
ax = axes[0]
ax.hist(init_scores, bins=50, alpha=0.5, label='Initial', edgecolor='white')
ax.hist(ref_scores, bins=50, alpha=0.5, label='Refined', edgecolor='white')
ax.set_xlabel('Matching Score')
ax.set_ylabel('Count')
ax.set_title('Score Distribution: Initial vs Refined')
ax.legend()

# Score improvement
ax = axes[1]
ax.boxplot([init_scores, ref_scores], labels=['Initial', 'Refined'])
ax.set_ylabel('Score')
ax.set_title('Score Comparison')

# Summary
ax = axes[2]
ax.axis('off')
stats = f'''CCA Refinement Results
{"="*40}

                Initial    Refined
Matches:     {len(init_scores):>10,}  {len(ref_scores):>10,}
Mean score:  {np.mean(init_scores):>10.4f}  {np.mean(ref_scores):>10.4f}
Median:      {np.median(init_scores):>10.4f}  {np.median(ref_scores):>10.4f}
Std:         {np.std(init_scores):>10.4f}  {np.std(ref_scores):>10.4f}

Score change: {np.mean(ref_scores) - np.mean(init_scores):+.4f}
'''
ax.text(0.1, 0.9, stats, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

In [ ]:
# Filter bad pivots
# NOTE: Using conservative filtering (20%) to retain more matches
# Increase filter_prop if too many low-quality matches remain
pivot_filter_prop = 0.2  # Remove bottom 20% (was 50%)

fusor.filter_bad_matches(
    target='pivot',
    filter_prop=pivot_filter_prop,
    verbose=True
)

print(f"\nFiltered {pivot_filter_prop*100:.0f}% of lowest-scoring pivots")

In [ ]:
# Visualize pivot filtering results
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Get remaining indices
remaining_idx = fusor._remaining_indices_in_refined_matching[0]
kept_scores = ref_scores[remaining_idx]
removed_scores = np.delete(ref_scores, remaining_idx)

# Score distributions: kept vs removed
ax = axes[0]
ax.hist(kept_scores, bins=30, alpha=0.7, label=f'Kept ({len(kept_scores)})', color='green', edgecolor='white')
if len(removed_scores) > 0:
    ax.hist(removed_scores, bins=30, alpha=0.7, label=f'Removed ({len(removed_scores)})', color='red', edgecolor='white')
ax.axvline(np.mean(kept_scores), color='darkgreen', linestyle='--')
ax.set_xlabel('Matching Score')
ax.set_ylabel('Count')
ax.set_title('Pivot Filtering: Kept vs Removed')
ax.legend()

# Summary pie chart
ax = axes[1]
sizes = [len(kept_scores), len(removed_scores)]
labels = [f'Kept\n{len(kept_scores):,}', f'Removed\n{len(removed_scores):,}']
colors = ['#2ecc71', '#e74c3c']
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
pct_removed = 100 * len(removed_scores) / (len(kept_scores) + len(removed_scores))
ax.set_title(f'Pivot Filter Results\n({pct_removed:.0f}% removed)')

plt.tight_layout()
plt.show()

print(f"Pivot filtering: {len(kept_scores):,} kept, {len(removed_scores):,} removed")
if len(removed_scores) > 0:
    print(f"Mean score - Kept: {np.mean(kept_scores):.4f}, Removed: {np.mean(removed_scores):.4f}")
else:
    print(f"Mean score - Kept: {np.mean(kept_scores):.4f}")


In [ ]:
# Propagate to all cells
fusor.propagate(
    svd_components1=min(40, n_rna_features - 1),
    svd_components2=None,
    wt1=0.7,
    wt2=0.7,
    verbose=True
)

In [ ]:
# Visualize propagation results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Get propagated matching
prop_match = fusor._propagated_matching[0]
prop_rows, prop_cols, prop_scores = prop_match

# Score distribution
ax = axes[0]
ax.hist(prop_scores, bins=50, edgecolor='white', alpha=0.7, color='purple')
ax.axvline(np.mean(prop_scores), color='r', linestyle='--',
           label=f'Mean: {np.mean(prop_scores):.3f}')
ax.set_xlabel('Matching Score')
ax.set_ylabel('Count')
ax.set_title('Propagated Matching Scores')
ax.legend()

# Coverage: pivot vs propagated
ax = axes[1]
pivot_rna = len(np.unique(ref_rows[remaining_idx]))
prop_rna = len(np.unique(prop_rows))
total_rna = rna_active.shape[0]

categories = ['Pivot\nMatches', 'Propagated\nMatches', 'Total\nRNA Cells']
values = [pivot_rna, prop_rna, total_rna]
colors = ['#3498db', '#9b59b6', '#95a5a6']
bars = ax.bar(categories, values, color=colors, edgecolor='white')
ax.set_ylabel('Count')
ax.set_title('Coverage Expansion')
for bar, val in zip(bars, values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
            f'{val:,}', ha='center', va='bottom', fontsize=10)

# Summary
ax = axes[2]
ax.axis('off')
prop_prot = len(np.unique(prop_cols))
total_prot = protein_active.shape[0]
stats = f'''Propagation Summary
{"="*40}

Propagated matches: {len(prop_scores):,}

RNA coverage:
  Pivot:      {pivot_rna:>8,} ({100*pivot_rna/total_rna:.1f}%)
  Propagated: {prop_rna:>8,} ({100*prop_rna/total_rna:.1f}%)
  Total:      {total_rna:>8,}

Protein coverage:
  Propagated: {prop_prot:>8,} ({100*prop_prot/total_prot:.1f}%)
  Total:      {total_prot:>8,}
'''
ax.text(0.1, 0.9, stats, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

In [ ]:
# Filter propagated matches
# NOTE: Using conservative filtering (10%) to maximize coverage
propagate_filter_prop = 0.1  # Remove bottom 10% (was 30%)

fusor.filter_bad_matches(
    target='propagated',
    filter_prop=propagate_filter_prop,
    verbose=True
)

print(f"\nFiltered {propagate_filter_prop*100:.0f}% of lowest-scoring propagated matches")

In [ ]:
# Get full matching
full_matching = fusor.get_matching(order=(2, 1), target='full_data')

print(f"\nMaxFuse Full matching results:")
print(f"  Total matches: {len(full_matching[0])}")
print(f"  Unique RNA cells: {len(np.unique(full_matching[0]))}")
print(f"  Unique Protein cells: {len(np.unique(full_matching[1]))}")
print(f"  Score range: [{min(full_matching[2]):.3f}, {max(full_matching[2]):.3f}]")

In [ ]:
# Apply score threshold to remove negative/low-quality matches
MIN_SCORE_THRESHOLD = 0.0  # Remove anti-correlated matches

# Get original stats
n_original = len(full_matching[0])
scores = np.array(full_matching[2])

# Filter by threshold
mask = scores >= MIN_SCORE_THRESHOLD
full_matching_filtered = (
    np.array(full_matching[0])[mask],
    np.array(full_matching[1])[mask],
    scores[mask]
)

n_filtered = len(full_matching_filtered[0])
n_removed = n_original - n_filtered

print(f"Score threshold filtering (min score >= {MIN_SCORE_THRESHOLD}):")
print(f"  Original matches: {n_original:,}")
print(f"  Removed (score < {MIN_SCORE_THRESHOLD}): {n_removed:,} ({100*n_removed/n_original:.1f}%)")
print(f"  Remaining matches: {n_filtered:,}")
print(f"")
print(f"Score statistics after filtering:")
print(f"  Min:    {np.min(full_matching_filtered[2]):.4f}")
print(f"  Max:    {np.max(full_matching_filtered[2]):.4f}")
print(f"  Mean:   {np.mean(full_matching_filtered[2]):.4f}")
print(f"  Median: {np.median(full_matching_filtered[2]):.4f}")

# Update full_matching to use filtered version
full_matching = full_matching_filtered

# Visualize what was removed
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

ax = axes[0]
if n_removed > 0:
    ax.hist(scores[~mask], bins=30, alpha=0.7, color='red', label=f'Removed ({n_removed:,})', edgecolor='white')
ax.hist(scores[mask], bins=30, alpha=0.7, color='green', label=f'Kept ({n_filtered:,})', edgecolor='white')
ax.axvline(MIN_SCORE_THRESHOLD, color='black', linestyle='--', linewidth=2, label=f'Threshold ({MIN_SCORE_THRESHOLD})')
ax.set_xlabel('Matching Score')
ax.set_ylabel('Count')
ax.set_title('Score Threshold Filtering')
ax.legend()

ax = axes[1]
if n_removed > 0:
    sizes = [n_filtered, n_removed]
    colors = ['#2ecc71', '#e74c3c']
    labels = [f'Kept\n{n_filtered:,}', f'Removed\n{n_removed:,}']
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax.set_title('Score Threshold Results')
else:
    ax.text(0.5, 0.5, f'All {n_filtered:,} matches\nkept (score >= {MIN_SCORE_THRESHOLD})', 
            ha='center', va='center', fontsize=12, transform=ax.transAxes,
            bbox=dict(boxstyle='round', facecolor='#2ecc71', alpha=0.3))
    ax.set_title('Score Threshold Results')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Investigate unmatched RNA cells
print("=" * 60)
print("UNMATCHED RNA CELL ANALYSIS")
print("=" * 60)

# Find matched and unmatched RNA cells
matched_rna_idx = np.unique(full_matching[0])
all_rna_idx = np.arange(rna_active.shape[0])
unmatched_rna_idx = np.setdiff1d(all_rna_idx, matched_rna_idx)

n_matched = len(matched_rna_idx)
n_unmatched = len(unmatched_rna_idx)
n_total = rna_active.shape[0]

print(f"\nRNA Cell Coverage:")
print(f"  Matched:   {n_matched:,} ({100*n_matched/n_total:.1f}%)")
print(f"  Unmatched: {n_unmatched:,} ({100*n_unmatched/n_total:.1f}%)")
print(f"  Total:     {n_total:,}")

# Save unmatched indices for further analysis
unmatched_rna_indices = unmatched_rna_idx

if n_unmatched == 0:
    print("\n*** All RNA cells matched! ***")
    print("No unmatched analysis needed.")
else:
    # Compare feature distributions: matched vs unmatched
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))

    # Shared feature means
    ax = axes[0, 0]
    matched_shared_mean = rna_shared[matched_rna_idx].mean(axis=1)
    unmatched_shared_mean = rna_shared[unmatched_rna_idx].mean(axis=1)
    ax.hist(matched_shared_mean, bins=30, alpha=0.6, label='Matched', color='green', density=True)
    ax.hist(unmatched_shared_mean, bins=30, alpha=0.6, label='Unmatched', color='red', density=True)
    ax.set_xlabel('Mean shared feature value')
    ax.set_ylabel('Density')
    ax.set_title('Shared Features: Matched vs Unmatched')
    ax.legend()

    # Shared feature variance
    ax = axes[0, 1]
    matched_shared_var = rna_shared[matched_rna_idx].var(axis=1)
    unmatched_shared_var = rna_shared[unmatched_rna_idx].var(axis=1)
    ax.hist(matched_shared_var, bins=30, alpha=0.6, label='Matched', color='green', density=True)
    ax.hist(unmatched_shared_var, bins=30, alpha=0.6, label='Unmatched', color='red', density=True)
    ax.set_xlabel('Variance of shared features')
    ax.set_ylabel('Density')
    ax.set_title('Feature Variance: Matched vs Unmatched')
    ax.legend()

    # Detection rate (non-zero features)
    ax = axes[0, 2]
    ZERO_VAL = -2.5  # From detection-aware normalization
    matched_detection = (rna_shared[matched_rna_idx] > ZERO_VAL + 0.1).mean(axis=1)
    unmatched_detection = (rna_shared[unmatched_rna_idx] > ZERO_VAL + 0.1).mean(axis=1)
    ax.hist(matched_detection, bins=30, alpha=0.6, label='Matched', color='green', density=True)
    ax.hist(unmatched_detection, bins=30, alpha=0.6, label='Unmatched', color='red', density=True)
    ax.set_xlabel('Fraction of detected features')
    ax.set_ylabel('Density')
    ax.set_title('Detection Rate: Matched vs Unmatched')
    ax.legend()

    # Active feature mean
    ax = axes[1, 0]
    matched_active_mean = rna_active[matched_rna_idx].mean(axis=1)
    unmatched_active_mean = rna_active[unmatched_rna_idx].mean(axis=1)
    ax.hist(matched_active_mean, bins=30, alpha=0.6, label='Matched', color='green', density=True)
    ax.hist(unmatched_active_mean, bins=30, alpha=0.6, label='Unmatched', color='red', density=True)
    ax.set_xlabel('Mean active feature value')
    ax.set_ylabel('Density')
    ax.set_title('Active Features: Matched vs Unmatched')
    ax.legend()

    # Active feature variance  
    ax = axes[1, 1]
    matched_active_var = rna_active[matched_rna_idx].var(axis=1)
    unmatched_active_var = rna_active[unmatched_rna_idx].var(axis=1)
    ax.hist(matched_active_var, bins=30, alpha=0.6, label='Matched', color='green', density=True)
    ax.hist(unmatched_active_var, bins=30, alpha=0.6, label='Unmatched', color='red', density=True)
    ax.set_xlabel('Variance of active features')
    ax.set_ylabel('Density')
    ax.set_title('Active Variance: Matched vs Unmatched')
    ax.legend()

    # Summary statistics
    ax = axes[1, 2]
    ax.axis('off')

    # Statistical comparison
    from scipy import stats
    t_stat_shared, p_shared = stats.ttest_ind(matched_shared_mean, unmatched_shared_mean)
    t_stat_detect, p_detect = stats.ttest_ind(matched_detection, unmatched_detection)

    summary = f"""Unmatched Cell Characteristics
{"="*45}

Shared feature mean:
  Matched:   {np.mean(matched_shared_mean):.4f}
  Unmatched: {np.mean(unmatched_shared_mean):.4f}
  p-value:   {p_shared:.2e}

Detection rate:
  Matched:   {np.mean(matched_detection):.2%}
  Unmatched: {np.mean(unmatched_detection):.2%}
  p-value:   {p_detect:.2e}

Interpretation:
"""

    if np.mean(unmatched_detection) < np.mean(matched_detection) - 0.05:
        summary += "  Unmatched cells have LOWER detection\n"
        summary += "  (sparse profiles harder to match)"
    elif np.mean(unmatched_shared_mean) < np.mean(matched_shared_mean) - 0.1:
        summary += "  Unmatched cells have LOWER expression\n"
        summary += "  (low signal harder to match)"
    else:
        summary += "  No clear pattern - may be cell type\n"
        summary += "  specific (check clustering)"

    ax.text(0.05, 0.95, summary, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

    plt.tight_layout()
    plt.show()

print(f"\nUnmatched RNA cell indices saved to 'unmatched_rna_indices'")
print(f"Use these indices to investigate in notebook 3 (visualization)")

In [ ]:
# Comprehensive final matching visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Score distribution
ax = axes[0, 0]
ax.hist(full_matching[2], bins=50, edgecolor='white', alpha=0.7, color='#2ecc71')
ax.axvline(np.mean(full_matching[2]), color='r', linestyle='--',
           label=f'Mean: {np.mean(full_matching[2]):.3f}')
ax.axvline(np.median(full_matching[2]), color='orange', linestyle='--',
           label=f'Median: {np.median(full_matching[2]):.3f}')
ax.set_xlabel('Matching Score')
ax.set_ylabel('Count')
ax.set_title('Final Matching Score Distribution')
ax.legend()

# Matches per RNA cell
ax = axes[0, 1]
rna_match_counts = np.bincount(full_matching[0], minlength=rna_active.shape[0])
ax.hist(rna_match_counts[rna_match_counts > 0], bins=30, edgecolor='white', alpha=0.7)
ax.set_xlabel('Protein matches per RNA cell')
ax.set_ylabel('RNA cells')
ax.set_title(f'Matching Density\n(mean: {np.mean(rna_match_counts[rna_match_counts > 0]):.1f})')

# Matches per Protein cell
ax = axes[0, 2]
prot_match_counts = np.bincount(full_matching[1], minlength=protein_active.shape[0])
ax.hist(prot_match_counts[prot_match_counts > 0], bins=30, edgecolor='white', alpha=0.7, color='orange')
ax.set_xlabel('RNA matches per Protein cell')
ax.set_ylabel('Protein cells')
ax.set_title(f'Reverse Matching Density\n(mean: {np.mean(prot_match_counts[prot_match_counts > 0]):.1f})')

# Coverage summary
ax = axes[1, 0]
n_rna_matched = len(np.unique(full_matching[0]))
n_prot_matched = len(np.unique(full_matching[1]))
categories = ['RNA', 'Protein']
matched = [n_rna_matched, n_prot_matched]
total = [rna_active.shape[0], protein_active.shape[0]]
x = np.arange(len(categories))
width = 0.35
bars1 = ax.bar(x - width/2, matched, width, label='Matched', color='#2ecc71')
bars2 = ax.bar(x + width/2, total, width, label='Total', color='#95a5a6', alpha=0.7)
ax.set_ylabel('Cells')
ax.set_title('Coverage Summary')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()
for bar, val in zip(bars1, matched):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'{val:,}', ha='center', va='bottom', fontsize=9)

# Score vs index (quality across matches)
ax = axes[1, 1]
sorted_scores = np.sort(full_matching[2])[::-1]
ax.plot(sorted_scores, linewidth=0.5)
ax.axhline(np.mean(full_matching[2]), color='r', linestyle='--', alpha=0.7)
ax.set_xlabel('Match rank')
ax.set_ylabel('Score')
ax.set_title('Sorted Match Scores')
ax.fill_between(range(len(sorted_scores)), sorted_scores, alpha=0.3)

# Final summary text
ax = axes[1, 2]
ax.axis('off')
coverage_rna = 100 * n_rna_matched / rna_active.shape[0]
coverage_prot = 100 * n_prot_matched / protein_active.shape[0]
summary = f'''MAXFUSE INTEGRATION SUMMARY
{"="*45}

Total matches: {len(full_matching[0]):,}

RNA cells:
  Matched: {n_rna_matched:,} / {rna_active.shape[0]:,} ({coverage_rna:.1f}%)
  Avg matches/cell: {len(full_matching[0])/n_rna_matched:.1f}

Protein cells:
  Matched: {n_prot_matched:,} / {protein_active.shape[0]:,} ({coverage_prot:.1f}%)
  Avg matches/cell: {len(full_matching[0])/n_prot_matched:.1f}

Score statistics:
  Mean:   {np.mean(full_matching[2]):.4f}
  Median: {np.median(full_matching[2]):.4f}
  Std:    {np.std(full_matching[2]):.4f}
  Min:    {np.min(full_matching[2]):.4f}
  Max:    {np.max(full_matching[2]):.4f}
'''
ax.text(0.05, 0.95, summary, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## Save Integration Results

Save integration outputs for use in subsequent visualization notebooks.

In [ ]:
# Save integration results to results directory
import os
import pickle
from datetime import datetime
import json as json_module

# Create results directory
results_dir = 'results/2_integration'
os.makedirs(results_dir, exist_ok=True)

# Save matching results (filtered by score threshold)
matching_data = {
    'rna_indices': full_matching[0],
    'protein_indices': full_matching[1],
    'scores': full_matching[2]
}
with open(f'{results_dir}/maxfuse_matching.pkl', 'wb') as f:
    pickle.dump(matching_data, f)
print(f'Saved MaxFuse matching: {len(full_matching[0]):,} matches')

# Save as CSV for easy inspection
matching_df = pd.DataFrame({
    'rna_idx': full_matching[0],
    'protein_idx': full_matching[1],
    'score': full_matching[2]
})
matching_df.to_csv(f'{results_dir}/maxfuse_matching.csv', index=False)

# Save unmatched RNA indices for further analysis
np.save(f'{results_dir}/unmatched_rna_indices.npy', unmatched_rna_indices)
print(f'Saved unmatched RNA indices: {len(unmatched_rna_indices):,} cells')

# Save normalized arrays used for integration
np.save(f'{results_dir}/rna_shared.npy', rna_shared)
np.save(f'{results_dir}/rna_active.npy', rna_active)
np.save(f'{results_dir}/protein_shared.npy', protein_shared)
np.save(f'{results_dir}/protein_active.npy', protein_active)
print(f'Saved normalized arrays')

# Save correspondence table
correspondence_df = pd.DataFrame(
    rna_protein_correspondence, 
    columns=['rna_gene', 'protein_marker']
)
correspondence_df.to_csv(f'{results_dir}/correspondence.csv', index=False)

# Save integration parameters
n_matched_rna = len(np.unique(full_matching[0]))
n_matched_prot = len(np.unique(full_matching[1]))
integration_params = {
    'timestamp': datetime.now().isoformat(),
    'method': 'maxfuse',
    'score_threshold': MIN_SCORE_THRESHOLD,
    'fusor_params': {
        'max_outward_size': max_outward,
        'matching_ratio': matching_ratio,
        'smoothing_method': fusor.method,
        'n_shared_features': rna_shared.shape[1],
        'cca_components': cca_components,
        'pivot_filter_prop': pivot_filter_prop,
        'propagate_filter_prop': propagate_filter_prop
    },
    'data_shapes': {
        'rna_cells': rna_active.shape[0],
        'protein_cells': protein_active.shape[0],
        'rna_active_features': rna_active.shape[1],
        'protein_active_features': protein_active.shape[1],
        'shared_features': rna_shared.shape[1]
    },
    'matching_stats': {
        'total_matches': len(full_matching[0]),
        'unique_rna_matched': n_matched_rna,
        'unique_protein_matched': n_matched_prot,
        'rna_coverage_pct': 100 * n_matched_rna / rna_active.shape[0],
        'protein_coverage_pct': 100 * n_matched_prot / protein_active.shape[0],
        'unmatched_rna_cells': len(unmatched_rna_indices),
        'mean_score': float(np.mean(full_matching[2])),
        'min_score': float(np.min(full_matching[2])),
        'max_score': float(np.max(full_matching[2]))
    }
}
with open(f'{results_dir}/integration_params.json', 'w') as f:
    json_module.dump(integration_params, f, indent=2)

print(f'\nAll outputs saved to {results_dir}/')
print(f'  - maxfuse_matching.pkl (pickle)')
print(f'  - maxfuse_matching.csv')
print(f'  - unmatched_rna_indices.npy')
print(f'  - rna_shared.npy, rna_active.npy')
print(f'  - protein_shared.npy, protein_active.npy')
print(f'  - correspondence.csv')
print(f'  - integration_params.json')
print(f'\nRun 3_visualization.ipynb next.')


In [ ]:
# Load the RNA data to check what genes are available
rna_adata = sc.read_h5ad('/home/smith6jt/maxfuse/results/1_preprocessing/rna_adata.h5ad')
rna_genes = set(rna_adata.var_names)

print(f"RNA data has {len(rna_genes)} genes")

# Check which of the new mappings have their genes in RNA data
print("\nChecking new mappings against RNA data:")
for protein, gene in new_mappings:
    if gene.startswith("Ignore"):
        print(f"  {protein}: {gene} (skipped)")
        continue
    
    # Handle multiple gene options (separated by /)
    gene_options = gene.split('/')
    found = any(g in rna_genes for g in gene_options)
    status = "✓ Found" if found else "✗ NOT FOUND"
    print(f"  {protein} -> {gene}: {status}")

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np

# Reload data
rna_adata = sc.read_h5ad('/home/smith6jt/maxfuse/results/1_preprocessing/rna_adata.h5ad')
protein_adata = sc.read_h5ad('/home/smith6jt/maxfuse/results/1_preprocessing/protein_adata.h5ad')
rna_genes = set(rna_adata.var_names)

print(f"RNA data has {len(rna_genes)} genes")
print(f"Protein data has {len(protein_adata.var_names)} proteins")

# The new mappings I added
new_mappings = [
    ("DAPI", "Ignore: DNA stain"),
    ("IAPP", "IAPP"),
    ("IDO1", "IDO1"),
    ("INS", "INS"),
    ("Ker8-18", "KRT8/KRT18"),
    ("iNOS", "NOS2"),
    ("M2Gal3", "LGALS3"),
    ("B3TUBB", "TUBB3"),
    ("PCNA", "PCNA"),
    ("Granzyme B", "GZMB"),
    ("HLA-A", "HLA-A"),
    ("VISTA", "VSIR"),
    ("Pan-Cytokeratin", "Ignore: too many genes"),
    ("LAG3", "LAG3"),
    ("SST", "SST"),
    ("TCF-1", "TCF7"),
    ("TOX", "TOX"),
    ("Caveolin", "CAV1"),
    ("ICOS", "ICOS"),
    ("EpCAM", "EPCAM"),
    ("Keratin 5", "KRT5"),
    ("GCG", "GCG"),
    ("Beta-actin", "ACTB"),
    ("Bcl-2", "BCL2"),
    ("MPO", "MPO"),
    ("Iba1", "AIF1"),
    ("SOX2", "SOX2"),
    ("TP63", "TP63"),
]

# Check which of the new mappings have their genes in RNA data
print("\nChecking new mappings against RNA data:")
not_found = []
for protein, gene in new_mappings:
    if gene.startswith("Ignore"):
        print(f"  {protein}: {gene} (skipped)")
        continue
    
    # Handle multiple gene options (separated by /)
    gene_options = gene.split('/')
    found_genes = [g for g in gene_options if g in rna_genes]
    
    if found_genes:
        print(f"  {protein} -> {gene}: ✓ Found ({', '.join(found_genes)})")
    else:
        print(f"  {protein} -> {gene}: ✗ NOT IN RNA DATA")
        not_found.append((protein, gene))

print(f"\n{len(not_found)} proteins have genes NOT in RNA data")

In [ ]:
# Let's fully simulate the notebook's matching logic
conversion_df = pd.read_csv('/home/smith6jt/maxfuse/data/protein_gene_conversion.csv', encoding='utf-8-sig')

proteins_in_adata = list(protein_adata.var_names)

# Build a lookup dictionary (case-insensitive)
protein_to_gene = {}
for _, row in conversion_df.iterrows():
    protein_to_gene[row['Protein name'].lower()] = row['RNA name']

# Try to match each protein
matched_pairs = []
unmatched = []

for marker in proteins_in_adata:
    marker_lower = marker.lower()
    
    if marker_lower in protein_to_gene:
        rna_name = protein_to_gene[marker_lower]
        
        # Skip "Ignore" entries
        if rna_name.startswith('Ignore'):
            unmatched.append((marker, f"Ignored: {rna_name}"))
            continue
        
        # Handle multiple gene options
        gene_options = rna_name.split('/')
        found_gene = None
        for gene in gene_options:
            if gene in rna_genes:
                found_gene = gene
                break
        
        if found_gene:
            matched_pairs.append((found_gene, marker))
        else:
            unmatched.append((marker, f"{rna_name} (not in RNA data)"))
    else:
        unmatched.append((marker, "Not in conversion table"))

print(f"Total proteins: {len(proteins_in_adata)}")
print(f"Matched pairs: {len(matched_pairs)}")
print(f"Unmatched: {len(unmatched)}")

print("\nUnmatched proteins:")
for prot, reason in unmatched:
    print(f"  {prot}: {reason}")

In [ ]:
# Check for specific genes that should be common
check_genes = ['MS4A1', 'FOXP3', 'CDH2', 'IAPP', 'NOS2', 'SST', 'GCG', 'MPO', 'SOX2', 'TP63']

print("Checking for genes in RNA data:")
for gene in check_genes:
    found = gene in rna_genes
    # Also check for case variations
    case_variations = [g for g in rna_genes if g.upper() == gene.upper()]
    print(f"  {gene}: {'✓ Found' if found else '✗ Not found'} {case_variations if case_variations else ''}")

# Search for partial matches
print("\nSearching for partial matches:")
for gene in check_genes:
    matches = [g for g in rna_genes if gene.lower() in g.lower()]
    if matches:
        print(f"  {gene}: {matches[:5]}")  # Show first 5 matches

In [ ]:
# Let's check how many genes were in the raw data vs filtered
# Load the lognorm version (before any subsetting for HVGs)
rna_lognorm = sc.read_h5ad('/home/smith6jt/maxfuse/results/1_preprocessing/rna_adata_lognorm.h5ad')

lognorm_genes = set(rna_lognorm.var_names)
print(f"rna_adata.h5ad: {len(rna_genes)} genes")
print(f"rna_adata_lognorm.h5ad: {len(lognorm_genes)} genes")

# Check if the missing genes are in the lognorm version
print("\nChecking if missing genes exist in rna_adata_lognorm:")
for gene in check_genes:
    found_hvg = gene in rna_genes
    found_lognorm = gene in lognorm_genes
    print(f"  {gene}: HVG={found_hvg}, lognorm={found_lognorm}")

In [ ]:
# Load the raw RNA data to check for these genes
from scipy.io import mmread
import gzip

# Load raw gene names
with gzip.open('/home/smith6jt/maxfuse/data/raw_feature_bc_matrix/features.tsv.gz', 'rt') as f:
    raw_features = [line.strip().split('\t') for line in f]

raw_gene_ids = [f[0] for f in raw_features]
raw_gene_names = [f[1] for f in raw_features]

print(f"Raw RNA data has {len(raw_gene_names)} genes")

# Check for the missing genes
missing_genes = ['MS4A1', 'FOXP3', 'CDH2', 'IAPP', 'NOS2', 'SST', 'GCG', 'MPO', 'SOX2', 'TP63']

print("\nSearching for missing genes in raw data:")
found_indices = {}
for gene in missing_genes:
    if gene in raw_gene_names:
        idx = raw_gene_names.index(gene)
        found_indices[gene] = idx
        print(f"  {gene}: ✓ Found at index {idx}")
    else:
        # Check case variations
        matches = [(i, g) for i, g in enumerate(raw_gene_names) if g.upper() == gene.upper()]
        if matches:
            idx, name = matches[0]
            found_indices[gene] = idx
            print(f"  {gene}: ✓ Found as '{name}' at index {idx}")
        else:
            print(f"  {gene}: ✗ Not in raw data")

In [ ]:
# Load the raw matrix to check expression
print("Loading raw matrix...")
raw_matrix = mmread('/home/smith6jt/maxfuse/data/raw_feature_bc_matrix/matrix.mtx.gz').tocsc()
print(f"Raw matrix shape: {raw_matrix.shape} (genes x cells)")

# Check expression for each missing gene
print("\nExpression stats for missing genes in RAW data:")
print(f"{'Gene':<10} {'Cells>0':>10} {'% cells':>10} {'Total UMI':>12} {'Max UMI':>10}")
print("=" * 55)

for gene in missing_genes:
    idx = found_indices[gene]
    gene_counts = raw_matrix[idx, :].toarray().flatten()
    n_cells_expressing = (gene_counts > 0).sum()
    pct_cells = 100 * n_cells_expressing / raw_matrix.shape[1]
    total_umi = gene_counts.sum()
    max_umi = gene_counts.max()
    print(f"{gene:<10} {n_cells_expressing:>10,} {pct_cells:>9.3f}% {total_umi:>12,.0f} {max_umi:>10,.0f}")

In [ ]:
# Now check how many of these expressing cells passed QC filtering
# The cell filter keeps cells with: 100 <= UMI < 100000, 200 <= genes < 8000, MT% < 39.5

# Load barcodes
with gzip.open('/home/smith6jt/maxfuse/data/raw_feature_bc_matrix/barcodes.tsv.gz', 'rt') as f:
    raw_barcodes = [line.strip() for line in f]

# Calculate per-cell metrics on raw data
print("Calculating cell QC metrics...")
cell_total_counts = np.array(raw_matrix.sum(axis=0)).flatten()
cell_n_genes = np.array((raw_matrix > 0).sum(axis=0)).flatten()

# MT genes
mt_genes = [i for i, g in enumerate(raw_gene_names) if g.startswith('MT-')]
mt_counts = np.array(raw_matrix[mt_genes, :].sum(axis=0)).flatten()
pct_mt = np.where(cell_total_counts > 0, 100 * mt_counts / cell_total_counts, 0)

# Apply same filters as preprocessing
MIN_COUNTS, MAX_COUNTS = 100, 100000
MIN_GENES, MAX_GENES = 200, 8000
MAX_MT_PCT = 39.5

cell_pass_qc = (
    (cell_total_counts >= MIN_COUNTS) &
    (cell_total_counts < MAX_COUNTS) &
    (cell_n_genes >= MIN_GENES) &
    (cell_n_genes < MAX_GENES) &
    (pct_mt < MAX_MT_PCT)
)

print(f"Cells passing QC: {cell_pass_qc.sum():,} / {len(cell_pass_qc):,}")

# Check each gene - how many expressing cells pass QC?
print("\nExpression in QC-passed cells:")
print(f"{'Gene':<10} {'Raw cells':>10} {'QC cells':>10} {'Filtered?':>12}")
print("=" * 45)

for gene in missing_genes:
    idx = found_indices[gene]
    gene_counts = raw_matrix[idx, :].toarray().flatten()
    
    # Cells expressing this gene
    expressing = gene_counts > 0
    n_raw = expressing.sum()
    
    # Of those, how many pass QC?
    n_qc = (expressing & cell_pass_qc).sum()
    
    filtered = "YES" if n_qc < 3 else "no"
    print(f"{gene:<10} {n_raw:>10,} {n_qc:>10,} {filtered:>12}")

In [ ]:
# Check protein detection rates for the "missing" markers
missing_proteins = ['IAPP', 'E-cadherin', 'iNOS', 'FOXP3', 'CD20', 'SST', 'GCG', 'MPO', 'SOX2', 'TP63']

print("Protein detection rates in CODEX data:")
print(f"{'Protein':<15} {'Detection %':>12} {'Mean expr':>12}")
print("=" * 42)

for prot in missing_proteins:
    if prot in protein_adata.var_names:
        expr = protein_adata[:, prot].X.flatten()
        detection = (expr > 0).mean() * 100
        mean_expr = expr.mean()
        print(f"{prot:<15} {detection:>11.1f}% {mean_expr:>12.2f}")
    else:
        print(f"{prot:<15} Not found in protein_adata")

In [ ]:
# Check the CODEX data source
import os

# List data files to identify tissue
data_files = os.listdir('/home/smith6jt/maxfuse/data/')
print("Data files:")
for f in sorted(data_files):
    print(f"  {f}")

# Check the CODEX filename pattern
codex_files = [f for f in data_files if f.endswith('.tsv') or 'codex' in f.lower() or 'CC' in f]
print(f"\nCODEX-related files: {codex_files}")

In [ ]:
# Check the 6551_cells.csv to understand the tissue
cells_df = pd.read_csv('/home/smith6jt/maxfuse/data/6551_cells.csv', nrows=5)
print("Columns in 6551_cells.csv:")
print(cells_df.columns.tolist()[:20])  # First 20 columns

# Check if there's tissue info in the preprocessing notebook params
import json
params_file = '/home/smith6jt/maxfuse/results/1_preprocessing/preprocessing_params.json'
if os.path.exists(params_file):
    with open(params_file) as f:
        params = json.load(f)
    print(f"\nPreprocessing params:")
    print(json.dumps(params, indent=2))

In [ ]:
# Get the list of RNA genes we need to protect (from conversion table)
conversion_df = pd.read_csv('/home/smith6jt/maxfuse/data/protein_gene_conversion.csv', encoding='utf-8-sig')

# Extract all gene names (handling multiple options separated by /)
protected_genes = set()
for rna_name in conversion_df['RNA name']:
    if pd.isna(rna_name) or rna_name.startswith('Ignore'):
        continue
    for gene in rna_name.split('/'):
        gene = gene.strip()
        if gene:
            protected_genes.add(gene)

print(f"Protected genes from conversion table: {len(protected_genes)}")

# Check which of these exist in raw data
raw_gene_set = set(raw_gene_names)
protected_in_raw = protected_genes & raw_gene_set
print(f"Protected genes found in raw RNA data: {len(protected_in_raw)}")

# The missing ones
missing_protected = protected_genes - raw_gene_set
if missing_protected:
    print(f"\nProtected genes NOT in raw data ({len(missing_protected)}):")
    for g in sorted(missing_protected)[:10]:
        print(f"  {g}")

In [ ]:
# Demonstrate the fix: filter genes but preserve marker genes
# This is what needs to go in notebook 1_preprocessing.ipynb

# Reload raw matrix for demonstration
print("Demonstrating marker-preserving gene filter...")

# Standard filter: min_cells=3
standard_gene_counts = np.array((raw_matrix > 0).sum(axis=1)).flatten()
standard_keep = standard_gene_counts >= 3
print(f"Standard filter (min_cells=3): {standard_keep.sum():,} genes")

# Find indices of protected genes in raw data
protected_indices = set()
for gene in protected_in_raw:
    if gene in raw_gene_names:
        protected_indices.add(raw_gene_names.index(gene))

print(f"Protected marker genes: {len(protected_indices)}")

# Combined filter: keep if (min_cells >= 3) OR (is protected marker)
protected_mask = np.zeros(len(raw_gene_names), dtype=bool)
protected_mask[list(protected_indices)] = True

combined_keep = standard_keep | protected_mask
print(f"Combined filter (standard OR protected): {combined_keep.sum():,} genes")

# How many marker genes are rescued?
rescued = protected_mask & ~standard_keep
print(f"\nMarker genes rescued by protection: {rescued.sum()}")

# Show which ones
rescued_genes = [raw_gene_names[i] for i in range(len(raw_gene_names)) if rescued[i]]
print(f"Rescued genes: {rescued_genes}")

In [ ]:
# Test the marker-preserving filter logic
# Simulate what the updated notebook cell will do

# We already have rna_adata loaded with 33538 genes (raw)
# Let's simulate the full pipeline

from scipy.io import mmread
import gzip

# Reload raw data
print("Loading raw data...")
rna_mtx = mmread('/home/smith6jt/maxfuse/data/raw_feature_bc_matrix/matrix.mtx.gz')
with gzip.open('/home/smith6jt/maxfuse/data/raw_feature_bc_matrix/features.tsv.gz', 'rt') as f:
    raw_features = [line.strip().split('\t') for line in f]
raw_gene_names = [f[1] for f in raw_features]

rna_adata_test = ad.AnnData(rna_mtx.T.tocsr(), dtype=np.float32)
rna_adata_test.var_names = raw_gene_names
rna_adata_test.var_names_make_unique()

# Calculate QC metrics
rna_adata_test.var['mt'] = rna_adata_test.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(rna_adata_test, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

print(f"Raw: {rna_adata_test.shape}")

# Apply cell filters (same as notebook)
sc.pp.filter_cells(rna_adata_test, min_counts=100)
sc.pp.filter_cells(rna_adata_test, min_genes=46)
rna_adata_test = rna_adata_test[rna_adata_test.obs['total_counts'] < 92731, :].copy()
rna_adata_test = rna_adata_test[rna_adata_test.obs['n_genes_by_counts'] < 7066, :].copy()
rna_adata_test = rna_adata_test[rna_adata_test.obs['pct_counts_mt'] < 39.5, :].copy()

print(f"After cell filters: {rna_adata_test.shape}")

# Now test marker-preserving gene filter
conversion_df = pd.read_csv('/home/smith6jt/maxfuse/data/protein_gene_conversion.csv', encoding='utf-8-sig')

protected_genes = set()
for rna_name in conversion_df['RNA name']:
    if pd.isna(rna_name) or str(rna_name).startswith('Ignore'):
        continue
    for gene in str(rna_name).split('/'):
        gene = gene.strip()
        if gene:
            protected_genes.add(gene)

print(f"\nProtected genes from conversion table: {len(protected_genes)}")

protected_in_data = protected_genes & set(rna_adata_test.var_names)
print(f"Protected genes found in filtered RNA data: {len(protected_in_data)}")

# Calculate cells per gene
n_cells_per_gene = np.array((rna_adata_test.X > 0).sum(axis=0)).flatten()

standard_keep = n_cells_per_gene >= 3
protected_mask = np.array([g in protected_in_data for g in rna_adata_test.var_names])
combined_keep = standard_keep | protected_mask

rescued_mask = protected_mask & ~standard_keep
n_rescued = rescued_mask.sum()
rescued_gene_names = [g for g, r in zip(rna_adata_test.var_names, rescued_mask) if r]

print(f"\nStandard filter (min_cells=3): {standard_keep.sum():,} genes")
print(f"Combined filter (standard OR protected): {combined_keep.sum():,} genes")
print(f"Marker genes rescued: {n_rescued}")
print(f"Rescued genes: {rescued_gene_names}")

In [ ]:
# Apply the filter and check how many protein-gene pairs we get
rna_adata_test = rna_adata_test[:, combined_keep].copy()
print(f"Final RNA data with marker preservation: {rna_adata_test.shape}")

# Now simulate the integration matching
proteins_in_adata = list(protein_adata.var_names)
rna_genes_new = set(rna_adata_test.var_names)

# Build lookup
protein_to_gene = {}
for _, row in conversion_df.iterrows():
    protein_to_gene[row['Protein name'].lower()] = row['RNA name']

# Try to match
matched_pairs_new = []
unmatched_new = []

for marker in proteins_in_adata:
    marker_lower = marker.lower()
    
    if marker_lower in protein_to_gene:
        rna_name = protein_to_gene[marker_lower]
        
        if pd.isna(rna_name) or str(rna_name).startswith('Ignore'):
            unmatched_new.append((marker, f"Ignored: {rna_name}"))
            continue
        
        gene_options = str(rna_name).split('/')
        found_gene = None
        for gene in gene_options:
            if gene.strip() in rna_genes_new:
                found_gene = gene.strip()
                break
        
        if found_gene:
            matched_pairs_new.append((found_gene, marker))
        else:
            unmatched_new.append((marker, f"{rna_name} (not in RNA data)"))
    else:
        unmatched_new.append((marker, "Not in conversion table"))

print(f"\n{'='*60}")
print(f"COMPARISON: Before vs After marker preservation")
print(f"{'='*60}")
print(f"Before (standard min_cells=3): 47 matched pairs")
print(f"After (marker-preserving):     {len(matched_pairs_new)} matched pairs")
print(f"Improvement:                   +{len(matched_pairs_new) - 47} pairs")

print(f"\nUnmatched proteins ({len(unmatched_new)}):")
for prot, reason in unmatched_new:
    print(f"  {prot}: {reason}")

In [ ]:
# Examine protein intensity distributions to understand background
import numpy as np
import pandas as pd

# Load the protein data
protein_adata = sc.read_h5ad('/home/smith6jt/maxfuse/results/1_preprocessing/protein_adata.h5ad')

print("Protein intensity statistics:")
print(f"{'Marker':<20} {'Min':>10} {'5%':>10} {'Median':>10} {'95%':>10} {'Max':>10}")
print("=" * 75)

for marker in protein_adata.var_names[:15]:  # First 15 markers
    vals = protein_adata[:, marker].X.flatten()
    print(f"{marker:<20} {vals.min():>10.1f} {np.percentile(vals, 5):>10.1f} "
          f"{np.median(vals):>10.1f} {np.percentile(vals, 95):>10.1f} {vals.max():>10.1f}")

In [ ]:
# Test different arcsinh cofactors and their effect on detection
# The issue: with cofactor=5, even background values (5th percentile) become "detected"

print("Effect of different cofactors on background vs signal separation:")
print("=" * 80)

# Pick a few representative markers
test_markers = ['IAPP', 'CD68', 'SMA', 'CD20', 'SST', 'GCG']
cofactors = [5, 50, 150, 500]

for marker in test_markers:
    if marker not in protein_adata.var_names:
        continue
    vals = protein_adata[:, marker].X.flatten()
    p5, p50, p95 = np.percentile(vals, [5, 50, 95])
    
    print(f"\n{marker}: raw 5%={p5:.1f}, median={p50:.1f}, 95%={p95:.1f}")
    
    for cofactor in cofactors:
        transformed = np.arcsinh(vals / cofactor)
        t5, t50, t95 = np.percentile(transformed, [5, 50, 95])
        
        # Estimate "detection" using a threshold
        # With arcsinh, values < ~0.5 could be considered background
        pct_above_05 = (transformed > 0.5).mean() * 100
        pct_above_1 = (transformed > 1.0).mean() * 100
        
        print(f"  cofactor={cofactor:>3}: arcsinh 5%={t5:.2f}, med={t50:.2f}, 95%={t95:.2f} | "
              f">0.5: {pct_above_05:.0f}%, >1.0: {pct_above_1:.0f}%")

In [ ]:
# Let's also try background subtraction approach
# Subtract per-marker background (e.g., 5th or 10th percentile) before arcsinh

print("Background subtraction + arcsinh approach:")
print("=" * 80)

for marker in test_markers:
    if marker not in protein_adata.var_names:
        continue
    vals = protein_adata[:, marker].X.flatten()
    
    # Background subtraction options
    bg_5 = np.percentile(vals, 5)
    bg_10 = np.percentile(vals, 10)
    
    # Subtract background, clip to 0
    vals_sub5 = np.clip(vals - bg_5, 0, None)
    vals_sub10 = np.clip(vals - bg_10, 0, None)
    
    print(f"\n{marker}: raw bg(5%)={bg_5:.1f}, bg(10%)={np.percentile(vals, 10):.1f}")
    
    # After background subtraction, use standard cofactor=5
    for name, vals_sub in [("sub 5%", vals_sub5), ("sub 10%", vals_sub10)]:
        transformed = np.arcsinh(vals_sub / 5)
        pct_zero = (vals_sub == 0).mean() * 100
        pct_above_05 = (transformed > 0.5).mean() * 100
        t50 = np.median(transformed[transformed > 0]) if (transformed > 0).any() else 0
        
        print(f"  {name}: {pct_zero:.0f}% zeros, {pct_above_05:.0f}% >0.5 (median of non-zero: {t50:.2f})")

In [ ]:
# Try data-driven cofactor: use median intensity as cofactor
# This normalizes each marker relative to its own scale

print("Data-driven cofactor approach (cofactor = marker median):")
print("=" * 80)

for marker in test_markers:
    if marker not in protein_adata.var_names:
        continue
    vals = protein_adata[:, marker].X.flatten()
    
    # Use median as cofactor
    cofactor_median = np.median(vals)
    # Use 75th percentile as cofactor  
    cofactor_p75 = np.percentile(vals, 75)
    
    print(f"\n{marker}: median={cofactor_median:.1f}, p75={np.percentile(vals, 75):.1f}")
    
    for name, cof in [("median", cofactor_median), ("p75", cofactor_p75)]:
        transformed = np.arcsinh(vals / cof)
        t5, t50, t95 = np.percentile(transformed, [5, 50, 95])
        pct_above_05 = (transformed > 0.5).mean() * 100
        pct_above_1 = (transformed > 1.0).mean() * 100
        
        print(f"  cof={name}: 5%={t5:.2f}, med={t50:.2f}, 95%={t95:.2f} | >0.5: {pct_above_05:.0f}%, >1.0: {pct_above_1:.0f}%")

In [ ]:
# Analyze this dataset's intensity distribution to derive a formula
# Goal: formula that gives ~150 for this data, ~5 for typical CyTOF data

# Get all protein values
all_protein_vals = protein_adata.X.flatten()

print("Current dataset intensity statistics:")
print(f"  Min: {all_protein_vals.min():.1f}")
print(f"  5th percentile: {np.percentile(all_protein_vals, 5):.1f}")
print(f"  10th percentile: {np.percentile(all_protein_vals, 10):.1f}")
print(f"  25th percentile: {np.percentile(all_protein_vals, 25):.1f}")
print(f"  Median: {np.median(all_protein_vals):.1f}")
print(f"  75th percentile: {np.percentile(all_protein_vals, 75):.1f}")
print(f"  95th percentile: {np.percentile(all_protein_vals, 95):.1f}")
print(f"  Max: {all_protein_vals.max():.1f}")

# Per-marker medians
marker_medians = [np.median(protein_adata[:, m].X.flatten()) for m in protein_adata.var_names]
print(f"\n  Median of marker medians: {np.median(marker_medians):.1f}")
print(f"  Mean of marker medians: {np.mean(marker_medians):.1f}")

# For CyTOF data, typical values after transformation are 0-10 range
# arcsinh(x/5) with x~25 gives ~2.3
# For this CODEX data, median is ~170, so arcsinh(170/5) = 4.2 (too compressed)
# We want arcsinh(170/150) = 0.99 (better dynamic range)

print("\n" + "="*60)
print("Formula derivation:")
print("="*60)
print("""
For arcsinh transformation to work well:
- Background (5th percentile) should map to ~0.1-0.3
- Signal (median-95th) should map to ~0.5-2.0

arcsinh(x/c) ≈ 0.2 when x/c ≈ 0.2, so c ≈ 5*x for 5th percentile

Proposed formula: COFACTOR = 5th_percentile * 5
""")

# Test the formula
p5_global = np.percentile(all_protein_vals, 5)
proposed_cofactor = p5_global * 5

print(f"5th percentile of all values: {p5_global:.1f}")
print(f"Proposed cofactor (5th_pct * 5): {proposed_cofactor:.1f}")

In [ ]:
# Per-marker cofactor calculation
# Formula: cofactor = 5th_percentile * k, where k is a scaling factor

print("Per-marker cofactor analysis:")
print("=" * 90)
print(f"{'Marker':<18} {'5th%':>8} {'Median':>8} {'95th%':>8} | {'cof=5':>8} {'cof=p5*5':>8} {'cof=p5*10':>8}")
print("-" * 90)

results = []
for marker in protein_adata.var_names:
    vals = protein_adata[:, marker].X.flatten()
    p5 = np.percentile(vals, 5)
    p50 = np.median(vals)
    p95 = np.percentile(vals, 95)
    
    # Test different cofactor formulas
    # Detection rate (>0.5 after arcsinh) for each
    for cof_name, cof in [("cof=5", 5), ("cof=p5*5", p5*5), ("cof=p5*10", p5*10)]:
        cof = max(cof, 1)  # Avoid division by zero
        transformed = np.arcsinh(vals / cof)
        det_rate = (transformed > 0.5).mean() * 100
        results.append({
            'marker': marker, 'p5': p5, 'p50': p50, 'p95': p95,
            'cof_name': cof_name, 'cofactor': cof, 'det_rate': det_rate
        })

# Print summary for first 15 markers
for marker in protein_adata.var_names[:15]:
    marker_results = [r for r in results if r['marker'] == marker]
    p5 = marker_results[0]['p5']
    p50 = marker_results[0]['p50']
    p95 = marker_results[0]['p95']
    det_5 = [r['det_rate'] for r in marker_results if r['cof_name'] == 'cof=5'][0]
    det_p5x5 = [r['det_rate'] for r in marker_results if r['cof_name'] == 'cof=p5*5'][0]
    det_p5x10 = [r['det_rate'] for r in marker_results if r['cof_name'] == 'cof=p5*10'][0]
    
    print(f"{marker:<18} {p5:>8.1f} {p50:>8.1f} {p95:>8.1f} | {det_5:>7.0f}% {det_p5x5:>7.0f}% {det_p5x10:>7.0f}%")

In [ ]:
# Try median-based formula: cofactor = median / k
# For CyTOF: median ~5-10, cofactor=5 means k=1-2
# For CODEX: median ~200, cofactor=150 means k~1.3

print("Testing median-based formula: cofactor = median / k")
print("=" * 90)
print(f"{'Marker':<18} {'Median':>8} | {'cof=med':>10} {'cof=med/2':>10} {'cof=med/1.5':>10}")
print("-" * 90)

for marker in protein_adata.var_names[:15]:
    vals = protein_adata[:, marker].X.flatten()
    p50 = np.median(vals)
    
    det_rates = {}
    for k, name in [(1, 'cof=med'), (2, 'cof=med/2'), (1.5, 'cof=med/1.5')]:
        cof = max(p50 / k, 1)
        transformed = np.arcsinh(vals / cof)
        det_rates[name] = (transformed > 0.5).mean() * 100
    
    print(f"{marker:<18} {p50:>8.1f} | {det_rates['cof=med']:>9.0f}% {det_rates['cof=med/2']:>9.0f}% {det_rates['cof=med/1.5']:>9.0f}%")

print("\n" + "="*60)
print("For CyTOF comparison (typical values):")
print("  If median ~5, cofactor=median gives cof=5 ✓")
print("  If median ~200, cofactor=median gives cof=200")
print("="*60)

In [ ]:
# Let's think about this differently
# The key insight: cofactor=5 works for CyTOF because typical values are ~0-50
# For this CODEX data, typical values are ~0-500+

# What if: cofactor = data scale indicator
# For CyTOF: 95th percentile ~50, so cofactor = 95th/10 = 5
# For CODEX: 95th percentile ~1500, so cofactor = 95th/10 = 150

print("Testing 95th percentile-based formula: cofactor = p95 / 10")
print("=" * 90)

all_markers_results = []

for marker in protein_adata.var_names:
    vals = protein_adata[:, marker].X.flatten()
    p5 = np.percentile(vals, 5)
    p50 = np.median(vals)
    p95 = np.percentile(vals, 95)
    
    # Formula: cofactor = p95 / 10
    cof = p95 / 10
    transformed = np.arcsinh(vals / cof)
    det_rate = (transformed > 0.5).mean() * 100
    
    # Also compute what arcsinh values the percentiles map to
    arcsinh_p5 = np.arcsinh(p5 / cof)
    arcsinh_p50 = np.arcsinh(p50 / cof)
    arcsinh_p95 = np.arcsinh(p95 / cof)
    
    all_markers_results.append({
        'marker': marker, 'p5': p5, 'p50': p50, 'p95': p95, 
        'cofactor': cof, 'det_rate': det_rate,
        'arcsinh_p5': arcsinh_p5, 'arcsinh_p50': arcsinh_p50, 'arcsinh_p95': arcsinh_p95
    })

print(f"{'Marker':<18} {'p95':>8} {'Cofactor':>10} | {'as(p5)':>8} {'as(p50)':>8} {'as(p95)':>8} | {'Det%':>6}")
print("-" * 90)
for r in all_markers_results[:15]:
    print(f"{r['marker']:<18} {r['p95']:>8.1f} {r['cofactor']:>10.1f} | "
          f"{r['arcsinh_p5']:>8.2f} {r['arcsinh_p50']:>8.2f} {r['arcsinh_p95']:>8.2f} | {r['det_rate']:>5.0f}%")

print("\n" + "="*60)
print("Summary:")
avg_det = np.mean([r['det_rate'] for r in all_markers_results])
avg_arcsinh_p5 = np.mean([r['arcsinh_p5'] for r in all_markers_results])
avg_arcsinh_p50 = np.mean([r['arcsinh_p50'] for r in all_markers_results])
print(f"  Average detection rate: {avg_det:.1f}%")
print(f"  Average arcsinh(5th pct): {avg_arcsinh_p5:.2f}")
print(f"  Average arcsinh(median): {avg_arcsinh_p50:.2f}")
print("\nFormula: COFACTOR = 95th_percentile / 10")
print("For CyTOF (p95~50): cofactor = 5")
print("For this CODEX (p95~1500): cofactor = 150")

In [ ]:
# Try more aggressive scaling
# Goal: arcsinh(background) < 0.5, arcsinh(high signal) ~ 2-3

print("Comparing formulas:")
print("=" * 100)

formulas = [
    ("p95/10", lambda p5, p50, p95: p95/10),
    ("p95/5", lambda p5, p50, p95: p95/5),
    ("p50", lambda p5, p50, p95: p50),
    ("p50*2", lambda p5, p50, p95: p50*2),
]

print(f"{'Marker':<15}", end="")
for name, _ in formulas:
    print(f" | {name:^18}", end="")
print()
print("-" * 100)

summary_stats = {name: [] for name, _ in formulas}

for marker in protein_adata.var_names[:12]:
    vals = protein_adata[:, marker].X.flatten()
    p5 = np.percentile(vals, 5)
    p50 = np.median(vals)
    p95 = np.percentile(vals, 95)
    
    print(f"{marker:<15}", end="")
    
    for name, formula in formulas:
        cof = max(formula(p5, p50, p95), 1)
        transformed = np.arcsinh(vals / cof)
        det_rate = (transformed > 0.5).mean() * 100
        arcsinh_p5 = np.arcsinh(p5 / cof)
        
        print(f" | cof={cof:>5.0f} det={det_rate:>3.0f}%", end="")
        summary_stats[name].append(det_rate)
    print()

print("\n" + "="*60)
print("Average detection rates:")
for name, rates in summary_stats.items():
    print(f"  {name}: {np.mean(rates):.1f}%")

In [ ]:
# The key insight: we need BOTH cofactor and threshold to be data-driven
# 
# For CyTOF (median ~5): cofactor=5 means values at median give arcsinh(1)=0.88
# So the implicit "detection" is roughly "above median"
#
# Let's use: cofactor = median, and detection threshold = arcsinh(1) ≈ 0.88
# This means "detected" = value > median (consistent interpretation)

print("Data-driven approach: cofactor = median, threshold = arcsinh(1)")
print("This means 'detected' ≈ 'above median intensity'")
print("=" * 90)

THRESHOLD = np.arcsinh(1)  # ≈ 0.88

print(f"\nThreshold: arcsinh(1) = {THRESHOLD:.3f}")
print(f"{'Marker':<18} {'Median':>10} {'Cofactor':>10} | {'Det (>0.5)':>12} {'Det (>0.88)':>12}")
print("-" * 80)

for marker in protein_adata.var_names[:15]:
    vals = protein_adata[:, marker].X.flatten()
    p50 = np.median(vals)
    
    cof = p50
    transformed = np.arcsinh(vals / cof)
    
    det_05 = (transformed > 0.5).mean() * 100
    det_088 = (transformed > THRESHOLD).mean() * 100
    
    print(f"{marker:<18} {p50:>10.1f} {cof:>10.1f} | {det_05:>11.0f}% {det_088:>11.0f}%")

print("\n" + "="*60)
print("Verification for CyTOF-like data:")
print("  If median=5, cofactor=5")
print("  arcsinh(5/5) = arcsinh(1) = 0.88 (at threshold)")
print("  So ~50% of cells detected (those above median) ✓")
print("\nFor this CODEX data:")
print("  median~200, cofactor=200") 
print("  arcsinh(200/200) = 0.88 (at threshold)")
print("  ~50% detection (those above median) ✓")

In [ ]:
# Better approach: cofactor = median, but use a lower threshold
# This way markers with actual signal will show higher detection
# 
# Key: arcsinh(0.5) ≈ 0.48, so threshold=0.5 means "above 0.5*median"

print("Final formula: cofactor = median, threshold = 0.5")
print("'Detected' means value > 0.52 * median (arcsinh(0.52) ≈ 0.5)")
print("=" * 90)

results_final = []

for marker in protein_adata.var_names:
    vals = protein_adata[:, marker].X.flatten()
    p50 = np.median(vals)
    
    # Cofactor = median
    cof = p50
    transformed = np.arcsinh(vals / cof)
    
    # Detection at threshold 0.5
    det_rate = (transformed > 0.5).mean() * 100
    
    # Also compute the arcsinh of key percentiles
    p5 = np.percentile(vals, 5)
    p95 = np.percentile(vals, 95)
    
    results_final.append({
        'marker': marker,
        'median': p50,
        'cofactor': cof,
        'det_rate': det_rate,
        'arcsinh_p5': np.arcsinh(p5/cof),
        'arcsinh_p95': np.arcsinh(p95/cof)
    })

print(f"{'Marker':<18} {'Median':>8} {'Det%':>8} | {'as(p5)':>8} {'as(p95)':>8}")
print("-" * 60)
for r in results_final[:20]:
    print(f"{r['marker']:<18} {r['median']:>8.1f} {r['det_rate']:>7.0f}% | {r['arcsinh_p5']:>8.2f} {r['arcsinh_p95']:>8.2f}")

print(f"\n{'='*60}")
print(f"Average detection rate: {np.mean([r['det_rate'] for r in results_final]):.1f}%")
print(f"Detection range: {min(r['det_rate'] for r in results_final):.0f}% - {max(r['det_rate'] for r in results_final):.0f}%")

print(f"\n{'='*60}")
print("FORMULA SUMMARY:")
print("  COFACTOR = median(marker_values)")
print("  For CyTOF (median~5): cofactor ≈ 5")
print("  For CODEX (median~200): cofactor ≈ 200")

In [ ]:
# Let's find a formula that maps:
# CyTOF (low values) -> cofactor ~5
# CODEX (high values) -> cofactor ~150

# Try different percentile-based formulas
print("Finding the right formula:")
print("Target: CyTOF (p75~50) -> cof=5, CODEX (p75~500) -> cof=150")
print("=" * 80)

# For this CODEX data, calculate what divisor gives ~150 for different percentiles
for pct in [50, 75, 90, 95]:
    pct_vals = [np.percentile(protein_adata[:, m].X.flatten(), pct) for m in protein_adata.var_names]
    avg_pct = np.mean(pct_vals)
    
    # What divisor gives cofactor=150 for this percentile?
    divisor_for_150 = avg_pct / 150
    # What divisor gives cofactor=5 assuming CyTOF has values ~10x lower?
    divisor_for_5_cytof = (avg_pct / 10) / 5  # Assuming CyTOF is 10x lower
    
    print(f"p{pct}: avg={avg_pct:.0f}, divisor for cof=150: {divisor_for_150:.1f}")

print("\n" + "="*60)
print("Proposed formula: COFACTOR = 75th_percentile / 3")
print("="*60)

# Test this formula
for marker in protein_adata.var_names[:15]:
    vals = protein_adata[:, marker].X.flatten()
    p75 = np.percentile(vals, 75)
    cof = p75 / 3
    
    transformed = np.arcsinh(vals / cof)
    det_rate = (transformed > 0.5).mean() * 100
    
    print(f"{marker:<18} p75={p75:>8.1f} cof={cof:>6.1f} det={det_rate:>3.0f}%")

# Average cofactor
avg_cof = np.mean([np.percentile(protein_adata[:, m].X.flatten(), 75) / 3 for m in protein_adata.var_names])
print(f"\nAverage cofactor: {avg_cof:.0f}")
print(f"For CyTOF with p75~50: cofactor = 50/3 = {50/3:.0f}")

In [ ]:
# Think about it differently:
# The cofactor should be proportional to the BACKGROUND level, not signal
# Background = lower percentiles (5th, 10th)
# 
# For CyTOF: background ~1, cofactor=5 means 5x background
# For CODEX: background ~30, we want cofactor ~150, that's 5x background

print("Testing: COFACTOR = 10th_percentile * 5")
print("Rationale: cofactor should be ~5x the background level")
print("=" * 80)

results = []
for marker in protein_adata.var_names:
    vals = protein_adata[:, marker].X.flatten()
    p10 = np.percentile(vals, 10)
    cof = max(p10 * 5, 5)  # Minimum cofactor of 5
    
    transformed = np.arcsinh(vals / cof)
    det_rate = (transformed > 0.5).mean() * 100
    
    results.append({'marker': marker, 'p10': p10, 'cof': cof, 'det': det_rate})

print(f"{'Marker':<18} {'p10':>8} {'Cofactor':>10} {'Det%':>8}")
print("-" * 50)
for r in results[:20]:
    print(f"{r['marker']:<18} {r['p10']:>8.1f} {r['cof']:>10.1f} {r['det']:>7.0f}%")

avg_cof = np.mean([r['cof'] for r in results])
avg_det = np.mean([r['det'] for r in results])
print(f"\nAverage cofactor: {avg_cof:.0f}")
print(f"Average detection: {avg_det:.0f}%")
print(f"\nFor CyTOF with p10~1: cofactor = 1*5 = 5 ✓")
print(f"For this CODEX with p10~30: cofactor = 30*5 = {30*5}")

In [ ]:
# The issue: p10 varies too much between markers (3 to 300)
# Solution: use geometric mean of p10 and p50 as a balanced estimate

print("Testing: COFACTOR = sqrt(p10 * p50) (geometric mean)")
print("This balances background level with signal level")
print("=" * 80)

results = []
for marker in protein_adata.var_names:
    vals = protein_adata[:, marker].X.flatten()
    p10 = np.percentile(vals, 10)
    p50 = np.median(vals)
    
    # Geometric mean of background (p10) and typical signal (p50)
    cof = np.sqrt(p10 * p50)
    cof = max(cof, 5)  # Minimum cofactor of 5
    
    transformed = np.arcsinh(vals / cof)
    det_rate = (transformed > 0.5).mean() * 100
    
    results.append({'marker': marker, 'p10': p10, 'p50': p50, 'cof': cof, 'det': det_rate})

print(f"{'Marker':<18} {'p10':>8} {'p50':>8} {'Cofactor':>10} {'Det%':>8}")
print("-" * 60)
for r in results[:20]:
    print(f"{r['marker']:<18} {r['p10']:>8.1f} {r['p50']:>8.1f} {r['cof']:>10.1f} {r['det']:>7.0f}%")

avg_cof = np.mean([r['cof'] for r in results])
avg_det = np.mean([r['det'] for r in results])
det_range = (min(r['det'] for r in results), max(r['det'] for r in results))

print(f"\n{'='*60}")
print(f"Average cofactor: {avg_cof:.0f}")
print(f"Average detection: {avg_det:.0f}%")
print(f"Detection range: {det_range[0]:.0f}% - {det_range[1]:.0f}%")

print(f"\nFormula verification:")
print(f"  For CyTOF (p10=1, p50=5): cof = sqrt(1*5) = {np.sqrt(1*5):.1f}")
print(f"  For CODEX (p10=50, p50=200): cof = sqrt(50*200) = {np.sqrt(50*200):.0f}")

In [ ]:
# Try: COFACTOR = 25th_percentile * 1.5
# This should give ~5 for CyTOF and ~150 for CODEX

print("Testing: COFACTOR = p25 * 1.5")
print("=" * 80)

results = []
for marker in protein_adata.var_names:
    vals = protein_adata[:, marker].X.flatten()
    p25 = np.percentile(vals, 25)
    
    cof = p25 * 1.5
    cof = max(cof, 5)  # Minimum cofactor of 5
    
    transformed = np.arcsinh(vals / cof)
    det_rate = (transformed > 0.5).mean() * 100
    
    # Also check what arcsinh value the 5th percentile maps to
    p5 = np.percentile(vals, 5)
    arcsinh_p5 = np.arcsinh(p5 / cof)
    
    results.append({'marker': marker, 'p25': p25, 'cof': cof, 'det': det_rate, 'arcsinh_p5': arcsinh_p5})

print(f"{'Marker':<18} {'p25':>8} {'Cofactor':>10} {'as(p5)':>8} {'Det%':>8}")
print("-" * 60)
for r in results[:20]:
    print(f"{r['marker']:<18} {r['p25']:>8.1f} {r['cof']:>10.1f} {r['arcsinh_p5']:>8.2f} {r['det']:>7.0f}%")

avg_cof = np.mean([r['cof'] for r in results])
avg_det = np.mean([r['det'] for r in results])
avg_arcsinh_p5 = np.mean([r['arcsinh_p5'] for r in results])

print(f"\n{'='*60}")
print(f"Average cofactor: {avg_cof:.0f}")
print(f"Average detection: {avg_det:.0f}%")
print(f"Average arcsinh(5th pct): {avg_arcsinh_p5:.2f}")

print(f"\nFormula verification:")
print(f"  For CyTOF (p25~3): cof = 3 * 1.5 = {3*1.5}")
print(f"  For CODEX (p25~90): cof = 90 * 1.5 = {90*1.5}")

In [ ]:
# Final formula: COFACTOR = p25 * 2
# This should give:
# - CyTOF (p25~2-3): cofactor = 4-6 ≈ 5 ✓
# - CODEX (p25~75): cofactor = 150 ✓

print("FINAL FORMULA: COFACTOR = 25th_percentile * 2")
print("=" * 80)

results = []
for marker in protein_adata.var_names:
    vals = protein_adata[:, marker].X.flatten()
    p5 = np.percentile(vals, 5)
    p25 = np.percentile(vals, 25)
    p50 = np.median(vals)
    p95 = np.percentile(vals, 95)
    
    cof = p25 * 2
    cof = max(cof, 5)  # Minimum cofactor of 5
    
    transformed = np.arcsinh(vals / cof)
    det_rate = (transformed > 0.5).mean() * 100
    arcsinh_p5 = np.arcsinh(p5 / cof)
    
    results.append({
        'marker': marker, 'p25': p25, 'cof': cof, 
        'det': det_rate, 'arcsinh_p5': arcsinh_p5
    })

print(f"{'Marker':<18} {'p25':>8} {'Cofactor':>10} {'as(p5)':>8} {'Det%':>8}")
print("-" * 60)
for r in results[:20]:
    print(f"{r['marker']:<18} {r['p25']:>8.1f} {r['cof']:>10.1f} {r['arcsinh_p5']:>8.2f} {r['det']:>7.0f}%")

avg_cof = np.mean([r['cof'] for r in results])
avg_det = np.mean([r['det'] for r in results])
avg_arcsinh_p5 = np.mean([r['arcsinh_p5'] for r in results])
det_range = (min(r['det'] for r in results), max(r['det'] for r in results))

print(f"\n{'='*70}")
print(f"SUMMARY:")
print(f"{'='*70}")
print(f"  Average cofactor: {avg_cof:.0f}")
print(f"  Average detection: {avg_det:.0f}%")
print(f"  Detection range: {det_range[0]:.0f}% - {det_range[1]:.0f}%")
print(f"  Average arcsinh(background): {avg_arcsinh_p5:.2f} (below 0.5 threshold ✓)")

print(f"\n{'='*70}")
print("FORMULA VERIFICATION:")
print(f"{'='*70}")
print(f"  For CyTOF-like data (p25 ~ 2.5):  cofactor = 2.5 * 2 = 5  ✓")
print(f"  For this CODEX data (p25 ~ 75):   cofactor = 75 * 2 = 150 ✓")
print(f"\n  Formula: COFACTOR = percentile(marker, 25) * 2")